In [1]:
import ipywidgets as widgets
from IPython.display import clear_output
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Layout, Button, Box, FloatProgress
from ipyfilechooser import FileChooser


In [2]:
from research_aspect_classifier import ResearchAspectClassifier

import IPython
store = IPython.get_ipython().find_line_magic('store')

import re
import spacy
import scispacy
from scispacy.linking import EntityLinker

import ktrain
import csv

import pickle

import os
import zipfile

import shutil

import random

import numpy

In [3]:
#bio_nlp = spacy.load('en_ner_bionlp13cg_md')
#nlp_sentences = spacy.load('en_core_sci_lg')

#linker = EntityLinker(resolve_abbreviations=True, name="mesh")

In [4]:
#bio_nlp.add_pipe(linker)

In [5]:
#doc = bio_nlp("Myeloid derived suppressor cells (MDSC) are immature myeloid cells with immunosuppressive activity. They accumulate in tumor-bearing mice and humans  with different types of cancer, including hepatocellular  carcinoma (HCC)")

#print(doc.ents[5].label_)

#linker.umls.cui_to_entity[doc.ents[5]._.umls_ents[0][0]]

## GLOBAL VARIABLES

In [4]:
test_uids = ['ca5twlo7', '9b5cuyae', 'kabpuznc', 'un6u1suu', 'pgyuk9z0', '8cq8p3iy', 'chmnogt6', 'qxso58u3', '1o7vxcvv', 'g3m7ksrs', 'w4i0q6rm', '3dk9wnw2', 'd7jmakcj', 'fo3wl3jy', '78rr4pz5', '04pp0o74', '31wrnk3m', 'a7zith7z', 'i9ret7bw', '2sz7j89r', 'po5o6996', '1m3bqw9s', 'ml673d5s', '8qcy0nl1', 'ks8brz5c', '7hjmwnhr', 'izyjo3r4', '2af5c1da', 'vkoawl96', 'uy97jjyi', 'qw05apnf', 'fb4hm4b0', 'aurhahaq', 'a5deq2a7', 'htbgb8r3', '4ifwiisy', '4g6wq80m', 'fu73pzgg', 's28lfwtj', '99jlejuv', 'uv4fvlos', 'pjr185mj', '20muep3u', '8xg6nn5x', 'kc546u59', 'qdeaw0uz', 'bnjupaik', 'n4dgqo73', '9vtauu52', 'sijh2ouj', 'aklipjm4', '13ym1n0k', 'r64d3fv8', 'ekc2zs4i', '4qmajxtb', '43jmvu7o', 'xo1f35jg', 'qfdp392f', '4ipuy7su', 'wrh27ban', '5icvvvp1', 'mm2iwtf6', '3yxrqa22', 'mq5xq46v', 'a8gfr4fa', '67h9z5uu', 'vlx22si0', 'bs3bwzah', 'ei1nkcmf', 'e5u1ymz9', 'edelrr0u', 'xj9md751', '55yf5o1z', 'gy9roon3', 'fzhj0ec4', 'zpmlb143', 'el283fej', 'cjb0daps', 'nf20zqcj', '4wk8gsli', 'btl03892', '6u1zo0f9', 'oihyz5v7', '0bm5yj05', 'yvboonk4', '3c34eufb', 'uxd3fspc', 'dkl82wip', '0qb5ik3o', 'vg9btpqr', 'tua114fw', '5k8zonn5', '3qjhfyuq', 'hhdrfok3', '7air498p', 'qh8ljx56', 'ashhxj6e', 'c0zd7pig', 'ybc1hovk', '6y1gwszn', 'vdrd74nz', 'vt8se423', 'velr8y30', 'ssa5rzd5', '61b4h5wv', 'il5aq8py', '52bkp7dp', 'tzsy86x0', '8j31wu4l', '9887fsv2', 'gmdu5adf', 'awy3aans', '5jemq8nm', 'azqj5mbk', 'g55ftypv', 'vq3axp9l', 'vwq7l4hn', 'g6sf0u9s', 'e5iua619', 'l3rq1er3', '3ryondlt', 'g6r83g0p', 'o3eokh4t', 'uqm35x4b', 'niq3hosc', 'vsinwqnr', 'upu5j0h6', 'bl8bdrcm', 'hq5ckpyp', 'o92a0z3w', 's3nk9kfq', 'b7ez2e3b', 'dv1wgdoi', 'euaenlnt', 'i2tnwxec', '79m2t343', '8ul87tm4', '7zxhlh68', '4nqum43e', '0qa93woh', 'wjzv2z3v', 'm55d7z3v', 'ecmg8kti', 'yaxcbje2', 'vbrv1wop', 'nhfjs12s', '05i5j6js', 'ix3mi83u', 'agm6xi21', 'kxaosvyc', 'ti5kd3yn', '63izqxxl', 'v4wbjcn1', 'un14oynk', '44h511ln', 'udq1aqwg', '09hws3r2', '95m0gbef', 'vqke48ur', 'e8iirjov', 'se3hgzuc', 'li6wpp3d', 'ou8lm4yl', '1yf1ae1d', '4w1vzd61', 'hq33z9n5', 'kmy4hdiu', 'svksysdu', '7ftztaqn', '5gkp9tlt', '8kym977f', 'hf2unn4e', '41najczp', 'grx4gx95', '7skrmp0e', 'x5ou9406', 'fdkbuw6e', '96vl3k1l', '4k8js9v2', 'sp6o6h31', '9p7hqk1u', '81hecvgz', 'jov2pj6l', 'x4hwygcr', 'yrab9oxr', 'k04i71ky', 'x5nfp25w', 'nhh5ze3a', 'koyl8v4p', 'bj637hih', '56347zmj', '9bg3t8rx', 'qj7lho9r', '1gpzrft6', 'r57rgg5n', 'ba1j3pyu', 'qx7ua9us', 'vpdjemou', 'pjeef9s5', 'fnx1l7ht', 'vw9ewpbf', 'dt396imz', '4tjxcpmd', 'mqa4m0lq', 'yw8a1ssd', '6iwfg3m1', 'he10rnk1', 'sy8tasri', '5ifw4xd6', 'kpsanhfj', '9mg9xqvc', '0tj6zv5q', 'iwjy6mmn', '8s5s30u8', '2c85mc7i', 'kpicvu3y', 'rndu5ake', '7xmpq0b5', 'hbutstjm', '78hbvvk9', 'vxi3uttf', 'l2goopd1', '1o85f1vt', '237zu9ak', 'nvp1qamf', 'x2k1qb0x', '0j1jdd1w', 'xp773fnp', '404tng58', 'v02fs28k', 'ra0n1dny', 'ls07rfx4', 'cp5ud4w8', 'l6p47erg', 'krx4hjmg', 'msr78qwy', 'ugaygkz9', 'g1k3zlax', '6q0zp4bh', '9d71k6qc', 'bawzonq1', '5hpbjkft', 'x81i5365', '6uccgmo8', 'hz5ozj90', 'qp7hx44u', '80c471y3', 'an2za6vj', '3i1ege5f', 'e5grlsht', 'kyjxrthe', 'f8u66hbt', 'ich6h3w6', 'bzhwvvjm', 'jjgiiifa', '29vtxz89', 'utng9v48', 'cxuq0e0j', '3o0n1sc1', '89y8u99u', 'yggcf7q2', '7z7cl0st', 'qob7k5ch', 'u9k5n8cx', 'd9dnugod', 'bbi15xvf', 'r3672xqt', 't0hs063s', 'bqtms06t', 'up9jeeiv', 'jsnzbuee', 'naodc0t9', 'joic5enl', 'gx5b4mds', 'cluz4ms6', '203sih0e', '78ajtcz9', 'rsvzbh06', 'titvzo31', 'nj645gz8', 'u4mn7jyx', 'tjjuokec', 'xim1c3bj', '3ug05pa9', '5b38y993', 'ux94i16y', '6x33a6g6', 'uvva8804', 'y626ov37', 'a6lcgwb9', 'xrkiuq1s', 'ciuo5m7w', 'pw8w6tga', '5bxynnhr', 'li6uaq0m', 'j6v0zaa7', 'jxutbxso', 'bpv7p7zo', 'u9vv0no1', '0sow1im8', '0u6nnv0u', 'mkcj6p15', 'z50cgfos', 'zcmzpat5', 'xfp32cx7', 'n1lj0vyg', '55z267ys', 'f0u4cvcc', 'wav2tfmx', 'chv8gwo1', 'k3vzcbca', '4wn3lx70', 'bp52vb9o', 'uvg61uba', '2s5xd1oc', 'zixj2u7c', '6uanhe2p', 'i9r2kjvf', 'kwzm0mod', 'vufygjcc', 'wlds1tl8', 'jpz2pr4v', 'lpy7mh95', 'uez5dnxr', '79gozr1p', 'pcyfx7t4', 'h917gbrt', 'hrujptoy', 'ujvg3u3k', '14by5ly6', 'skpwg8pt', '9w5skoi1', 'veodk758', 'm4g0ik11', 'uobknrx7', 'nwtfs1t9', 'whxuhjcq', '0jmpd2tj', 'q8gfs9j6', 'zvq9corx', 'tbjt97rc', 'y16rn8kc', '5zzqt1vp', '52c7myio', 'majelie8', 't9ryky6f', 'ch8rpn2d', 'wpucxp2a', 'xjfau2cb', 'xrgnt6l5', 'af986870', '91nj68tn', '85u6sdwi', 'ckvsn0ah', 'oqjl3hq2', '72vcn5b1', 'ale9el74', 'r31xh0fw', '9lgyuzoq', '7habtj4m', '8y4j510s', 'ns30mkif', 'ux40twpt', '6ycsf8wh', '26wlrh0n', 'lzews17o', 'ayy6lhoz', '8ydmba6q', 'lnwhscn3', 'ocnozlc2', 'ghi0ekm5', 'cj7arfwe', '4crdebgq', 'wevi00fk', 'owskhb2q', 'pdeqk6xb', 'ji113ctp', 'w34n45wb', 'fp9q8ayj', 'db0l0wja', 'c7weqc03', '0n6lb8yl', 'ysx2twu3', 'm9hkfyff', 'e4h47fo4', '3i5pvcgu', 'b8rrz7o2', '3lj3kzlx', 'ccg27nmf', 'o4sbsf8s', 'djv4syy4', 'isthr32p', 'l4gf8m9j', 'jxu3056n', 'pi6nvedq', 'qwxyuuz7', 'v829uve7', '5sy7imd1', '2hhi28t7', 'gtfx5cp4', 'r7qqeqpz', '535mb4sc', '3ndd6q6i', 'ydg1j3rq', '5ci5och4', '42ahqmjc', '5od0tegt', 'tnlalu6p', 'tedawrkd', 'kwq2y3il', 'eh768g0y', 'bsz7ezzi', 'z1837c72', 'mrhd4iph', 'j4ouhh73', '0z10ahnz', '3aozpdr3', 'bl1kksew', 'dj4a271g', '9wfk73kt', '0kr9sc16', '3ugudmx7', '7mpflh5c', '5mtbixnh', 'hq4715ex', 'xmxu12qx', '4vyluec3', 'okue1kso', 'rza8hn0k', 'hkdzyo0u', 'zakegxrg', '40mdql39', 'pijvvo2i', 'gr1m24hj', 'zxb7trqu', 'hlvod4j1', '42ywl8gp', 'gxbf17m8', 'p3qsn8di', 'vqqqip1m', 'zgre7vbo', '0j6th4k0', '2ud5hfiw', '2jgjb6uj', 'qpydnvpx', '2nksr3b9', 'ox9v8xen', 'n4v4n8d1', 'mv2tr5ha', '20zujaha', 'mvkj3pql', 'cjfm44ef', '87pqhnei', 'y50ou7gg', 'k5d3thvc', 's2b050g5', 'hayo8jm0', 'llde79k6', 'kc6dls2z', '3wm6wd5m', 'r6h015fa', 'ocsimm2r', '0fbmelx0', 'lwu5ioyw', 'em1ieatt', 'riem28zd', 'q4of4qpo', '30ihtibn', '846x9j9b', 'z86hc4zy', 'ry5boej6', 'nbcjv47q', 'byh9lipz', 'bmmmuvpt', 'zfe4lbr0', 'rjfoi9om', 'ota09hwf', '0qur0isb', 'raddzrns', 'lk8qfh90', 'g693adjd', '2x9s768c', '1b3pigtl', '7h1wg0oa', 'vl3j05bs', 'awieq3ke', 'ea6lya1c', 'tvxaxfqv', 'kehnbcqm', 'ogzs1lat', 'va8kg3kb', 'bv5un9bt', '9o7kpfe1', 'infbg7uw', 'kdbu4jbs', 'ryojd2b3', '4yvf683z', 'zror7j90', 'pn9voawj', 'sa4ol247', 'tss23gns', 'scvcck1e', '2gmx6sry', '9ie8mx4l', '2emwfx89', '73vle1nk', 'hdy4h13e', 'iglif10z', 'dgjhm82z', 'jn6ib8dk', 'b64ijw5o', '8jhubc2q', '0lq8ql6n', '0x7fx15p', '7rng9wra', '48lpb69q', '8hh3kbs7', 'h86cufyp', '41d7343o', 'i8hd2ojl', '1gmzpp5k', '5kvyj5ub', '9o2tk4x5', 'adnhf39a', '5imbomop', 'anxbfrx5', '281bqcu9', 'vw34eijx', 'btisdpn6', 't1dulbic', 'kmvq6bit', 'bl5d4xkc', 'kqicjw48', '605fayyq', 'skj6jmsg', 'vvji04xp', '74ojotqy', 't6ltbxqn', '8vlsz26g', '4d99ghrn', 'ynnxcrbv', 'wzd8lb6z', 'p8ah7o0f', '8dc52b9l', '2u3ret98', '9v4s0zsj', 'qvvtvky6', 'hqy67hx3', 'y992yuzp', '1dwsm3ie', 'cuzan8zn', 'a89s59ex', 'pyaqlxu6', 'vnf6kgv5', '7173mcdx', 'npk3mdbh', 'n11ya3e9', 'z6op9hyd', 'ucbr6ak3', 'bg4twnhu', 'atd4rb1r', '5go8mo4m', 'k2s6fqrj', 'vcqkmib1', 'etaiv90a', 'nvs3kz78', 'w7ad8i9x', '6nsigwce', 'fcrunf38', '53cwj3gg', 'di0zfm9g', 'jhraafm2', 'fsuab9bx', 'uo89h8z7', 'xay5aw7y', 'uqdh1q7a', '9fm1ewss', '3wob432t', '1i5mml5g', 'sa2dkxlv', 'bisnmjlr', 'l0n9gjhd', '6ezodchx', '4w5o8jfu', '29bsbc2p', 'd5zs687b', 'v1to1mrp', 'asqpi36g', 'pqpnlu4n', 'k1m72zs6', 'af2ucgq6', '7s7h1led', 'bfxw158b', 'r36wjakw', 'edpyn8fd', '6zqes3dz', 'sj5o16by', 'j1vqgxke', 'arblbs1w', 'jvdvuw9y', 'qxhm3eum', 'gxwwhnm3', 'kacjdolq', 'cv6ids8u', 'pzv7pd2p', 'zblitbo0', 'vuy5yec7', '5lehe3y2', 'd86edsrw', 'c2wzjbgb', 't88654wo', 'h5j4f0xq', 'wvwyhauw', '6ms9pnbw', 'prqrsb3c', 'mt7wlvqo', '134w3jv1', '6oqahl60', 'nc6tqv9v', 'y63qrtud', 'pfv7q4v6', '1hmzgwrw', 'v6q0vdmi', 'yznjzpos', '7uc6v0i1', 'ob3pdfon', 's67xk5tm', 'tibjmumz', '0hxan9rw', 'gnfsmby6', '6quksaqa', 'v4x2lzdg', 'k9h73l1k', 'vbjm87nc', 'dlvwwkhs', '8j8s285b', 'f5k1rusk', 'o76lmo65', 'eyfu2uwk', '42rqyukv', 'dbhida9b', 'tcyum2j3', 'uct8tcb2', 'vvcqu0m2', 'bctb0ewu', 'hsy8h1vb', 'fbox3a5q', 'o8lb6jeb', 'br8v7mgd', '4eo9nzy2', 'w465nsgc', '3fw8z027', '9wiho9xs', '6tvhegbx', 's2p2csrx', '3nhqx2qn', 'svhwtorf', 'ov5wmbgf', 'ggrydkg1', 'u3o4xu29', '1p48febv', 'x110yvrm', 'efoodt9i', 'qhz2kzy9', '3sxcf3q2', 'so4pxxht', 'go9d6k2o', 'xsjfjwou', 'dhpzd357', '9gh6tj7g', 'ypohyvfn', 'f9jr9kki', 'ftt9hcu3', '0zsjr9dc', 'gwvs39yx', 'bqfk0hf5', 'z7leh72x', 'mwfv98a0', '7zt3bc3a', 'f6566mvf', 'jdd1bl7e', 'guhv84jy', 'b36uadfk', '8ybdn1f8', 'keqago2s', 'fgpsly4f', 'todrdmvi', 'h81niamz', 'lj4mq31p', '0pyzxwqu', 'ph10txit', 'a3x3gmn9', 'tpsf05ca', 'rfgogqmu', 'kprj8wvi', 'vsf9x7hy', 'j8hked94', 'qaxde1qe', 'z1mix8ol', 'z6s4diei', 'c7jwizpj', 'lydqo5o6', 'hffga2mh', 'uoslzgnh', 'n71fq8q4', '566pbuic', 'ojzbt809', 'r9srhj4q', 'k2f667sc', 'iiouxqlz', 'mq6qjs2s', 's8pmmrp3', 'o5i8pfkv', 'yb5rxrii', 'zghq7mcg', 'i1seyphn', 'e1zgqqqu', '2i7jzc18', 'zjg64lua', 'l2r6uprq', 'tizbmvlz', 'ks7j1z29', '2d9fzvpq', 'ztcij4wb', 'kygnk4uj', 'uzmj8zki', 'azt39js9', 'tuzmu7p5', 'm69ceq2t', 'xu6t5kwz', '6s22uhu9', 'qc9p1k60', '96kegg2i', 'xk1vczn4', 'slsru2pc', 'ot9ob2ih', 'bmjrmu56', 'h2oyd3jd', '82yqoza6', '7x4bzss2', 'mvrc6xg9', '2geq9mpr', 'qokws0b4', 'h3ngmi4c', 'vf7qlzir', '5pqpf433', 'bvi7cieo', 'txliznrt', 'of3exw6a', '3cz34tk4', 'b5vnzylk', 'w0sguu0t', 'oxntnusq', 'ek7udwrw', 'cx4slb0x', 'berlwmth', 'c8wtlon5', 'u74jut63', 'l0xzoa0u', 'v2ef7283', '5pg0a0y3', 'gfznl5ec', 'xk48los6', 'kw7dso40', 'xw244riy', 'jwolz4qk', 'il50pj73', '1oi9aw0i', '8dv6hkl7', 'acvlw7lr', 'ezvsi0ox', 'irkroegh', 'p8cyn487', '1usamo31', '47au8krf', 'g3gxpt22', 'apt12wrx', 'bfuxmae7', 'l2ew9ol0', 'n15v8z4t', 'ba78l50o', 'gux02vzu', 'yxizepar', 'f4lr91vi', 'b3byozz0', 'u4doukk7', 'k50k5qhe', '3z1tlv7t', 'radetygy', 'iyy8o4ii', '6lcb8nsy', '09nvausz', 'wxvy01cg', 'uklt524a', 'a038jilw', 'x9ehr5zj', '266mt81l', 'kk39ywfa', 'cv20koaq', 'ed736tf4', '0c1ja5bd', 'neba2o7n', 'ewqkb53p', 'fbggujfj', '8z6qs443', 'q4pp9v2m', 'vtro6oen', '68vbo6ap', '1fzvno87', '172tl3fb', 'sbry393q', 'm67acbzd', 'sym43q4k', 'uxehz43v', '1edrmcke', '7ozknn8m', 'k66fen94', '6dijw282', 'im346y3b', '1o1avuii', '51g3uspv', 'r5gdxdl2', 'hhbd0ket', 'xm9yj5ua', 'uywxy6z2', '3d7w79ik', 'cyvgy3x6', 'axymuyev', 'ipznmc07', 'plfjkp5f', 'nimvpuau', 'nhc7klsy', 'ymf2o6p4', 'odxm2fgt', 'gjchjvz7', 'dqo6loy2', '8mhoklev', 'j6wr8ud9', 'mm1qbw88', '040w9ba1', 'l5y7wmsc', 'qhft0vvj', 'rjnvetck', 'f6p9i951', 'ofccakya', '99pxfsnx', 'qi5tqrhs', 'hdccw28f', 'b0uxizrl', 'sct14hzu', 'lv3d1c95', 'k9a3awfc', 'ntf6pneq', '71tvb9kn', 'z6gjf4w1', 'v9686u0q', 'o07j4kio', 'ko8lyoob', 'eizfr544', 'r4ger87o', 's25do8cb', 'cjbfe2y5', 'f9j13fsc', 'ib5cyon0', 'pou2z1ll', 'g6mb63f7', 'm2mgfol3', 'j2utiy77', 'c4nlyjiz', 'b20veiaw', 'o1fx5aeo', 'synpltu8', '8dpdfsu4', 'galvv4bs', '4rv6tbks', 'cx3lf0dx', 'ds9425vs', 'qltomztx', 'b6dksrbx', '02am7l4t', '63ppn1rj', 'qrfx165d', 'de7sfccy', 'ygijxuq1', 'x2k9apav', 'km5z3hpf', '61i4jjys', 'sc3f67xo', 'b0zqydit', 'cwk0c241', 'rj62polb', 'xui1famo', '8nw2adde', 'zr59mmrt', 'v53cc5jk', '328t6fp9', 'rh6luded', 'kjzz4tuq', 'uxujtakt', 'c3ehp7na', 'jz9kknuq', 'a0zpwfzf', 'c4dvf3u6', 'vst2v5tw', 'wjsvunae', '3i6pnheg', 'rcbon30h', '13ee4d5j', 'vsyuv42l', 'oyk8js16', 'yn71y3gr', 'wic7n6ia', 'ofmrcot2', '2ufwx5jp', 'cc2n8lpm', '8s8gezpz', 'z0b94oxh', 'u25usodk', '9x9stq1g', 'o0uw6qtf', 'yhdupo32', 'd2f7kfmv', '153tpjks', '89milkae', '28z3kgbd', 'z7rgaufl', 'm7cqlzbh', 'znce5set', '0662jz20', 'zfprrxt4', 'mpc7l1z8', 'j5mxdxi8', '3qgxp3tw', '2g0obbqp', 'eerqrqwk', 'hd3h2s1p', 'l3evdou4', 'wfnsibyg', 'fc86sjec', 'g5rvho9p', 'o315kfol', '518e2x3h', '1cyk0ajn', 'n4nnmlv1', 'huyogbis', 'q88x155m', 'ej3naer9', '51dtwa7n', '5wiq0npx', 'um0j2ee1', 'hrcf2udq', 'gycxlzwy', '9pv527un', 'z0l81bzx', 'nnzhntvc', 'cp2yeqh0', 'icl1t9d6', 'likez6fh', 'uvs1rhep', '450z8gej', 'x4pr3hcl', '06mzvg5n', 'nklq6iuw', 'zc7u3g34', '0soyxeoy', 'crmhwuka', 'm5c4t88g', 'z8dt5x1b', 'imw9llwm', 'sjub0mwv', 'oh0dy2vm', 'mi44ysym', 'bdmu9cz2', '5qsgmxhu', 'c4mfprzc', 'i4k2yvqk', 'cyem9epc', 'g6xr5vxp', 'fmmm8itx', 'jdvvegp9', '7gfs3s2p', 'ciizdlhu', 'nyjt2wz1', '6w4bw709', 'iod72cbe', 'ryuelzbj', 'd4jczi45', '6q0a5jl0', '377y9aqx', 'ryuyfkyh', '75imqul9', 'n3gzni77', 'nkdnfij1', '6f52k1r0', '1i8s9r5k', 'r4l4kbk6', '1y79i676', 'hpm75mh0', 'dy79bxsq', '1p5uog38', 'hl1wcsnh', '7j4gbmf7', 'y2p97hqa', '81mtsuie', 'tsv0rcr3', 'igw5qyj2', 'm9curjp4', 'b2x4r4dp', 'xbys1l32', 'be35k2yq', 'hqhvj7sq', 's00465sy', 'o7r7a56v', 'dwjimnil', 'ojhc7peo', 'y3dv7k19', '6w8jun1e', '9xcwqcpw', 'zixnvehs', 'bai4n7m9', 'tjxktruz', '3of2vhrd', '02jls39a', '6p411mgz', 'b5kw494g', 'i91sqwbi', 'bwkad2s0', 'yyvatdph', 'nv3gbpg7', 'z9u4lbyx', 'yglo31o2', 'z5c8k4yo', 'vx3eqoak', '4je9wl1o', 'hjw1940x', 'glg0xsvb', 'gvobf5j7', 'bwummgm1', 'kh23ancj', 'luuq2bd9', '2buuykdp', 'bhc0t1zc', 'zl9pp2h3', 'hqhc6b9z', 'oblat93a', 'fl4flnn6', '6it2j4s0', 'i9c2mrqb', 'xf3t4npv', 'ybwwt75j', 'ahxnhutv', 'hfzbiucf', '6bq3g51s', 'ndxs60jq', '8r45mjwq', '5rokgdx7', 'kvnprkl3', '76j74nbc', 'cqt858wr', 'qz91j3gm', 'cfpd45qf', 'bis2arbu', 'i2zdk1m1', '5wazfkey', 'bn7zbjfy', 'umi1u2d5', 'r7fcfvl9', 'jtsxpjhc', 'roa7l6c0', 'r70neeiz', 'qy0s6uyr', '9jdg7sot', '43y51lh4', 'f047dvb4', 'a8xt4jlw', 'n5gw1ol3', '6baw4hmt', 'na3ccs16', 'wtvfu5ph', '2paegrlk', '3vhmgord', 'v15q3sys', '3ly4f8dy', 'e9bz9tup', 'hwjsddvd', 'jay3o4nj', 'qtr76n96', 'kw96caqn', '3wdquuzy', 'ypfgvq9x', 'az7oilzc', 'behe7zl6', 'v0zpdwr4', 'fdwpep90', 'z4teq35q', 'pruvl2l4', 'vb91zuyo', 'gei2r6aj', 'hotn7qha', '5u3578ws', 'krsuxi6x', 'd1qi4mpu', '03ct019o', '7ssjrvn6', 'bbolfmhw', 'aup1ygk2', 'q8d7rvnj', 'fprppp68', 'j20bdlzb', 'nd3v05ln', 'w224am1i', '6vpvzyd5', 'fnjt44oa', 'qlv0jttw', 'a5jqttue', 'nn7itiys', '633l8nlr', 'vpfj4c32', 'mw83uvhy', '0tkzjt1y', '7glb334h', 'f0dujuy3', 'h8tzyupj', 'd8l23zrr', '7agq980m', 't34qqe45', 'b6or1n6b', 'wfsq7vr5', '67t2q5ud', 'u1mrvjjf', 'kym7wtbm', 'k2wbjybi', '2bz78yl1', 'y40s45iv', '237dd3i0', 'vxzoyfyh', 'rxt75zex', 'h3etttzu', '1syjbpoa', 'nuadau49', '12q5iai3', 'njv30vtm', '9o0sjnml', '9h9u0ylr', 'pepeja37', '2vkarnw6', 'fgq6ueqk', 'cspf70b4', 'cd4sou1z', 'k0ko82ny', 'fhaklvlm', 'n0vi617l', '0ew54gmb', 'qwpya4ox', 'ge4dtrt3', 'q8pevs6u', '6lrawta5', 'bih9t439', 'g3pwguq4', '5522zjpq', 's7p88fzw', '6cgajmb8', 'aj3o436d', 'xql1cl04', 'xidrxd14', 't9jis9e6', 's5f1olt4', '0l01j65t', '9o1b32h3', '4dnuilqg', 'sgishpgc', '7kgnefo4', 't82jdpoj', '3dejrzqi', '1xycng32', '7qjg4d9q', 'kktoen1l', 'b5fhqc9y', 'pv68fb7d', 'ossmgzzn', 't7p01234', 'zsdnq14d', 'tkzm3esp', 'dgztaopl', '0edzhgnk', '6bdzut3d', 'd0ztc4pl', 'c4vtr18e', 'smtpjt6a', 'r3xacf6t', 'ntfa1tcs', 'xnj6agxt', 'ost4bgqm', 'lf1bbw5u', '1y11u5n4', 'hsczfea1', 'g2i96lmc', 'heui8rox', 'i378juhz', 'vwukiwe4', 'pb6o2i0u', 'dcugognz', 'njy99jh4', 'i5pl776p', 'txraboaj', '6iwqgbi9', 'v09tuzg7', 'v4e771kj', 'iawtjm9j', 'n8283y5e', '6dq8xx7c', 'djy7a4j3', 'td05l9ct', 'nazbpozh', '3vlc74a2', '3v9dpxk3', 'qx34vwbn', 'qtlq7a6l', 'sh0x5f9y', 'izrx07oi', 'plqe5esz', '4dgd5p8b', 'l0csf1wt', 'pipk6gjc', 'jbcfuk9a', '8ugsz6oj', 'y394rzbo', '9jl89mry', 'm0vqst1q', 'ej3gz2g0', '3p8a3ext', 'ljcr0xtm', 'jqy4y18a', 'tqpm3qj5', 'cgbc3l6l', 'p2cw15qq', 'v8vhx6rd', '30oay4nc', 'fovqhiaa', 'njmcrrgk', 'b56mrbr0', 'ivec5koq', 'ksintgvq', 'jd0dmp46', 'iy2rs47v', 'ylsqd0jq', '1m4milji', 'klwni1ui', '73tqvd72', '3merensk', 'n66qiqis', 'q0ietz55', 'ex9nh1yi', '9i3f6j8m', '8o6k2fzy', 'n3ky9302', 'wn2kohgm', 'sd8o79ed', 'mwu8812h', 'nzynerfu', 'cxiac4pm', 'j6tl5foo', 'pv7ijdnx', 'tgye33y5', '7mq9uau4', '35dt6vlt', 'bturl9cx', '4npwx3rf', 'r83bjxnc', 'z8p4cj03', 'uz4407as', 'kxodo1kw', '46yb41hq', 's6v6piqg', 'yba84x03', 'rteysj8j', 'bqq85ch4', 'k8xcp5br', '1026xa7n', 'f3ljyaax', 'ms44e4og', 'ebr716xv', 'olx1zeet', '8ufjh70o', 'cyivgkq6', '9yx50c6j', 'k8s2wcxf', 't5p09le8', 'e6xh7qw7', 'dg2dx21s', 'yvjxw39o', 'l228rygw', '9edjqt8m', 'ojwbuf9n', '5cxb9ktc', 'v8ibiksx', '73oe5as9', '2zdg5dk8', 'ilx9j89c', 'vsj1pxah', '7sqd8le9', 'f87yct0g', 'ex4owfjy', '4df5l01l', 'nan1pxe2', 'gdkc7owu', 'hnsi6x8a', '1b7e0xus', 'm9im4pdi', '360ypqwa', 'jprbp1ua', '985d4fuj', 'b7jyyaah', '8mjk8v0h', '6583s7nz', '24i98di1', '7j8t2cn7', '96xerchf', 'fx4t0814', 'bv56r4r8', 'hia1me9i', 'bki4cv6q', 'hc0za6fh', 'sc3ebxgb', 'fe0vw9eo', 'wf2oxc3v', '3faqvwoo', '96sxxh4u', 'hyzv8ofq', '80htwxm0', '3lr0815p', 'm7vazpq9', 'y07c647k', '496p6sld', 'dgis935x', '4r65cn42', '9xesz6u4', 'mpak8as5', 'mbh3an2e', 'du65ztzu', 'kkdg5753', 's4owrmdd', 'sgissak3', 'x5zhuk7k', 'vvrznffy', 'bekyn1k2', 'buxqyld0', 'wayhy1b1', 'zafq7vqa', 'fg4302j0', 'dp81iknk', '9uabw6u5', 'yemg28ds', 'cuclkx06', '7ooajare', '1ui0p8eu', '0gegnxu1', '2x07eq37', 'jdofdaoi', 'd1crwbuz', 'nwt1whvv', 'yts6sxry', 'qb28o1jl', 'hrenm8b8', '0k2oc16y', 'y5a558og', 'dvps2hxf', 'l5u68vxe', 'kfxv5jiw', 'zpdnzwle', 'atbfc8uq', 'smyp9fyo', '42ocgz4f', 'vtj1kh8i', 'kb9uesnl', 'apz1afhp', 'yb4qre1t', 'rdz4hv0s', 'pvyiyeu3', 'dl25ns5c', 'h4suvtbj', '5ycxh1sq', '33c3mp1o', 'msut5gyc', 'tkq1tno5', '8zm237fm', 'kbfay9dz', 'jpxe6zgj', 'i758v1vb', 'be4bqv5n', 'yts3nm42', 'ihsr7bhp', 'yw3k78kg', 'frbwxfk7', 'n16pdrc4', 'czpjrozj', 'ezrrcsf3', 'te1bf690', 'j0ktzqx7', 'ka9j9585', 'd7krzqb4', 'vsg21t7n', 'qbk1x6kp', 'hzzmbrc7', 'v7sntcbk', 'c7d6fpc5', 'oxzxche3', 'd0tg9i90', 'id79l4iq', '18tn2v9q', 'fejngjnx', '6vna8hww', 't9wdeqch', 'tf0v2ojk', 'eaty59va', '6ex5wiho', 'b1xvnarf', 'id4oos0w', '2jiu9v0o', 'wueb5fif', 'mtljy3s6', '5mcsfb3v', 's5hl04cu', 'g9yw4df1', 'ig7dycds', 'p2wbky8x', 'vvorhzd9', 'ilbwrh0z', '8t0eks8g', 'spm4j5wy', 'an45l0vp', 'q7tmkebq', 'hriq38et', 'r829ang0', 'd78pmgrb', '2r7iym8p', 'uw7g3y71', '4kbd5b3a', 'x73q967x', 'fr1w955e', 'mujyq3kk', 'e1uqi3qv', 'w33e2078', '7l9x20g3', 'qd4dic30', 'jxnl4iqc', '76nnkkuz', 'pji6gicu', 'bgxxtfoi', '4oychsu6', '5fiwyjdl', 'e5auwisn', 'oaez0n5h', 'pm40ny87', '5fa43b1u', 'igoz2ei4', 'qd8kro09', 'cf2w4xy6', 'tevkop3i', 'rc18d6o5', 'w3iis2sd', 'ehifoeuq', 'h26zms3f', 'vy3zdj8p', 'tbsccwgx', '8cbws7lw', 'd0hdn8pm', 't90jkrn2', 'sb65piot', 'a7q27nky', 'fsxqtzc2', '3cz543oe', 'wg9jdny7', 'hkopopa6', '7o7zj1fk', 'kyoa5gsf', '0xtz23j5', '6al4k4mk', 'q5vvt1dk', '7vnhdcaf', 'l3a8g8g5', '070ok2lm', '1pcgw5ey', 'toq9wrg9', 'z88xcjsw', 'hnlcwa7m', '8jjd99u5', '0iydpg2t', 'ngywgodg', 'mg4c5zca', '6tbqw0st', 'dtnpj9gr', 'fuqjzgqk', 'dz6a8h3b', '0s3btp4p', 'rwh56zhg', 'ammlbi6s', 'rtyhmgb2', 'slaykbhi', '85pk9441', 'pfx1eh5b', '23jb5wkh', 'eb07zmn0', 'kqdwxo1y', 'iqo8optv', 'c2mpraux', '9l53c2dt', 'n45b06do', '2ukm42np', 'o78scael', 'jxat6ket', '6h60960n', 'p5ixvjd7', 'jf090vmy', 'mklu5uus', 'hqk337s0', 'scydncr9', 'uhmxkvit', 'r9ovq1o2', 'qijo070w', '0jivgcrk', 'knfxnvbc', 'op7lmnkj', 'ghbuv382', '6sdqe80x', '3cml1shh', '8ldbydv4', '0wb06qg9', 'u2m9vnto', 'b7puixqx', 'lfq1znzm', 'dpcrhlui', 'wqzabk43', '4kl6h1jr', 'sw34iotp', 'gl7lpjcy', 'itbtp3iv', 'rjugon21', 'ngjbvik6', 'i37aoq2p', 'bx6qixio', '0gopxxtv', '7umn0vkv', '9a8khssg', 'kkvd004i', 'eml8uilb', 'ebpam5lz', '0eq041t1', 'zt4zut37', 'qmck9pp9', 'vpih1wvs', '4br9oth9', 'cfi5zhgu', '475u7pp8', 'l294vx4q', '061nz14q', 'y3q2dlsd', 'wmgt2kgi', '8rqbvieo', 'g5dpqcxe', 'xvfl7ycj', 'eas75gr6', 'axjaf4kh', 'tbl50bp6', 'rv2akbj8', 'rw7ljtfl', '43eghdc9', 'nztgxnrp', '6ob3u2e2', '330muq8x', '7nnbn6wk', 'bif2rncq', 'r0grl2sc', 'a30v3ktv', 'sd4us2pt', 'jdwnogkb', 'r6h4bf79', '77dtxxd4', 'kejwsvbr', '5qg4nin3', '9x937eus', 'gci53wa2', '05xdfg6z', 'vjcrhwqz', 'hlsovsyw', 'w46uofz3', 'lbp1otn2', 'w6hgt0ql', 'v3e1zqpa', '0pq1r8pg', 'p7ldfrbv', 'ltdfbk4k', 'm2orsnzp', 'r9tsf1e1', 'qunw5hhy', 'zkz8e1vd', '824miiwu', 'ohrc50k1', '88m1hx9d', 'vpodtbjk', 'sop1yjv7', 'xnol6e6r', '2is7twtp', 'q5qkqipv', '3rv17inb', '5mwyzzz8', '56tmg38b', 'fdc5wnfn', 'y3ppfknx', 'uv7i4zve', 'h64gi0f5', 'omj41rv7', 'szpl96hz', 'wwidm522', 'vgqtcvoh', 'ne4e6nc0', 'zodxdjgr', '8fjoi4em', 'b5lcv77o', '4u1n3yhm', '6kv372ue', 'arxyu96p', 'y7nbshmh', 'cuemi94l', 'eoj9eo2b', '8u685f45', 'og86wdmf', 'bf1ts879', 'hpglok9u', 'oykk71cp', 'jo38hjqa', 'y3sdqckk', 'vf2lgoi8', '1hvihwkz', 'jxtch47t', 'ff8v6dth', 'dql9eo1h', 'figp5aan', '1xjgtj0t', 'cw48tg50', 't5mej3lc', 'tn7epi4t', 'y3fhubnc', 'r2ne1hju', '8508n0ex', '9p0dsyqx', 'e7ntocgf', 'w87tfl88', 'p9720aun', 'uw5qyyc4', 'bu08dx20', 'rs680nf9', 'gzzkngo0', 'm0pzyt13', 'wm83513h', 'fg003v35', '9k7ngjwl', 'f8cgocuz', '53elkc79', 'kjoi54ea', '6otr8zl3', '3eokzfbh', 'pwd7w9r4', 'rfni76sv', 'rhyxj0yk', '0nioym64', 'ar4n0jka', '54d3p49g', 'ai39ny9y', 'puoutldy', '8odfh9yd', 'j9mvulr0', 'erm705tg', '7kw6kn09', 'cb66129p', '4ptc6trf', 'wc7ijt5s', 'hn1shmog', 'mzya8d8b', '0ldvip21', '72s8wpla', 'klw8zufe', 'ond9q78k', 'pp2sln56', 'dr611nse', 'i2g66p03', 'wrmu94g3', 'oh972c6r', 'l7ymcyw7', 'glmni128', '85z0fc08', 'bz6xy94r', 'dz89mqp9', 't9j0f4h7', '3pdy5yy5', 'v4tck4ob', 'vpto0r3f', 'kp661rmo', 'j5nvo67b', 'a62k3lma', '0b4k73sh', '1hdm32a7', '6gsz1tuy', '0xqhm8a0', '6vhz1l2h', 'gu3idi0e', 'onqy21w5', 'xuuucwgj', 'w4jswggv', 'gidlrnu8', 'o3r0y6bz', 'iub6pis4', '9136us8q', '61uqnx1h', 'mkjogy6h', '2chle80y', '4rjp16zg', '9nrhgj0e', 'm3ln3nlg', 'jpi5jy06', 'l211d603', 'bga48qut', 'ncou5ns6', 'l95unj6g', '05a65aa3', 'warffg6c', 'vsgviu8z', '99yk3vgj', 'xwwu4eg1', 'zoakplwl', 'zmgh5pxo', 'k82feq3a', 'glpca9u0', 'sl7t5dou', 'xndojxqk', 'mfqlv3nh', 'wumtwdi5', '77ht5mh4', 'fw5au4ec', 'm8leqng5', 'v6vg7q2e', 'yl9zu2ww', 'ipa0vnzn', '42pjc0lo', 'dwt9xzmc', 'g4gvts0v', 'ggx8i0i7', '9hq39ytc', 'arpr2wfc', 'yvrr4h4i', 'k4vk11bv', 'jk28r07t', 'n48hbt6g', 'wl0wlxh6', 'dtlnjcqm', '9ybj3rur', '75sbxnsc', '0agobx70', 'e19cldqc', 'eep3oc2l', '1vifok85', 'hygpknnz', '2yd4wg9v', 'kp5rd0hq', 'hhcv8b13', 'amom9lbc', '9fzxqbdn', 'ddgoxape', 'gb3oxfak', '9ituqb3g', 'jnuxa85o', 'bfeq9i1m', 'hmksupw9', 'cn3bpmwj', '3x5j91s1', '1ksfcvi2', '2splemb7', '8t59flj5', '2leiv2nx', '2eawq8vq', '64mrflcr', '2f4hr49f', 'b84fwvmn', '8mhk2r4x', 'o0qzp5dw', '21pxsawn', 'ff55bny2', '900zxjow', 'mdome1mh', 'rp97ysy1', '9vt50ex0', 'pcsoe7yt', 'tv3jttfx', '61dv84dv', '96vextpa', '55mxt7i9', '9kr34sg1', 'b498mggt', 'igebmiet', '6wgegsa6', 'kfw8ulm3', 'ptw0bjdb', 'h1fdp3ty', '06r9yatd', 'krohs84t', 'lq0pqmcv', '41wjrjhw', '12dc47vm', '3qydeela', '40e3c0lq', 'xg4hdp4w', '9mo6h3l3', 'mkcmfbxx', 'gzy4yud3', '95wclabe', 'fezblpl8', 'ltdav7ic', 'l4f8c13w', '3rjwx61g', '24wyajfj', 'vsbjwsvu', 'ac6o97ee', 'ffgd2ets', 'rj9zftdg', 'mgag9p8c', 'z0q8wnrk', 's94ud6xx', 'f79z7o0e', 'nqiwzpmj', '38hst77r', 'pn11rt09', 'b2rvovj3', '4kw6hbtg', 'ixhrm10k', 'sz3xz0kt', 'nhgc6wcg', 'ytjw6ax8', '5ijrl5uf', 'imtzviau', '9lcdhbei', 'xundzch6', 'p58g4w4a', '6vwxm2t8', '3pribklz', 'pcekcvwr', '1yadpir1', 'nrpyq399', 'xjn5pf63', 'l93zev26', '7lfn387g', 'w37lfv7e', 'oik00pe7', 'lh1r2rsq', '48i49o1f', 'oncq8v6c', '5kyx6ycq', '0rvvcfwj', 'lnws9tzu', 'nh2jdvur', '4xi0akbo', 'o7wu8lbb', 'k9wl04je', 'hxn4bgec', 'jr98r6ih', 'u75q9xsd', 'csdjkq82', 'dggauxi7', '37o2gjo6', '2aggilzg', 'kux4q1g3', 'u9ia86qd', 'lghjiw5p', '191dn7w5', 'i449zk6p', 'ivfqa7w7', 'xc4se4tb', 'aikaxbzf', 't8z759t4', 'uhcj2tzg', '734exidf', '6gc7smqf', 'bqbj660w', 'xqqmtkyk', 'cjt7cqqm', 'tcp69kym', '30uacrvh', 'mfqwah23', 'mcek2295', '59zcy1qv', '6nkm9wds', 'mgd5zh91', 'y1809x4w', 'houxpagf', '95z6rnak', 'yieon37a', 'ikc2hxr5', 'vex39sc4', 'wti8qlnd', 'vgwedvaa', '3difto4b', '2lxs9laj', 'h2gwormv', 'rb71qtvb', 'i2maz8gg', 'u2owpi5e', 'dudf192x', 'bcgbdf84', 'wlntj2ra', 'gd8217va', 'rzda8x62', 'yem63gpe', 'omsc9a8k', 'md4ilfik', 'avqzykq2', 'rp4k9nvb', '93kxeb50', '40xpx6ej', 'qr4fhokp', 'q2b4ig1h', '19r2xwl2', 'bbg4kjb7', '7d76gq32', '9x0jyah2', 'pqvtp38a', 'pz5juaje', 'hg7ukwxz', '59qwi9k5', '0r12f0h3', 'xwzsszhb', 'yb7cquxn', 'yedzgyew', 'ffr8gcky', 'ilimt1tf', 'x9dml7mb', 'zijxe016', '7fkaj1h8', 'er0zo50v', 'b4hrl9g8', 'v555a2ll', 'npoje09j', 'cy8sh4cs', '4a829n10', 'mjay109c', 'ckhoiqcr', '5qcwdnu2', '8p8i4z18', '0oxwfm2a', '8hawgdid', 'e8ohc25w', '5ums22je', '7l7kw7f9', 'nag78skl', 'qjfujfwm', 'p1a662h5', 'rppviv5v', 'dbpt4dhr', 'wsn7y3wr', 'q99n2gqp', '9m2wtmuc', '6v763k4s', '4b37noy7', 'm23k11iy', '6l55soxu', '9it9pgq1', 'x1km7nfn', 'ex6kdxy0', '8xmokvxu', 'njky7v5x', 'm4z1xsvp', 'xlzrknzu', '4fouhwq0', '2g7gmn3n', 'aa8dp7mc', '60khdt8c', '9iad2y0w', 'y2lf7ssp', 'ibqqoye1', '0xymzkzn', 'vdemfx6z', 'queiig1g', 'y88h1jeo', '3erodkv9', 'sp6019yz', '8r0lvico', 'pn75xam2', 'idnaulj6', 'j4vv4ki0', '9h4pq7up', '3cnun58x', 'hsgsuor2', '1nczw70h', 'ovdt4lmn', '2jy2hjwy', 'yhfxoc3q', '0qmtv1kc', 'bus3axay', 'kgrqnmfr', 'iz4f4rdn', 'odp78c0p', 'w8l37wq3', '0ymwnf22', 'xyhx54ql', 'wsfu2xnh', 'uw3r6q45', 'x5vzxcxf', '2mohptl2', 'm6l3cztp', 'p190ka5q', 'ttbur07i', 'nol2n8fo', '2akskxmn', 'ifpu2nyu', 'd32mexty', 'bkcf0spt', 'smkpp89u', 'c74f69qh', 'dawlrbbc', 'k3r166iv', 'hksotkne', 'oneqjp5h', 'jpkxjn6e', 'peqyn05d', '5x4j2b8l', '8jwc6erf', 'yo3xidz1', 'j1mvr9r9', 'csxyayof', 'hc9ucl4a', 'q6xsdx3t', 'mx1a3xht', 'tkobxmir', 'dxz02ig8', 'w5is1tbo', '0ne6q2x5', 'weow1ct0', '24bail03', 'pjoqahhk', 'd639fno3', 'nt61m84v', '94rf4hxq', 'cc28wixb', 'hfz19qmg', 'ktgda1ox', '7g7yt23q', 'doly7iku', 'w9ee5dlo', '01l8c1ln', 'huidye73', 'aydr2e2q', 'f19xgyfe', '54xhx29c', '64c7mn9b', 'fs2gwmg1', '924ded7t', '8j83wqhy', 'pyjpx849', 'gak0pvw0', 'vm1pedqm', 'wdj4wuax', 'sj59rzlh', 'ubuetsi3', 'j5v77tof', 'njei8dj8', 'euzh18tq', 'ens89ijc', '5083xxfp', 'a726gtyr', 'scszt7ic', 'twfmit6n', 'ri6776pc', '8kp07f7i', 'ge0iryuw', 'z0chxoi0', 'xvbjbzu0', 'x9ahwdpi', '9w9a3rs9', 'gxy2njaa', 'ipnt6g0i', 'kifsj94h', 'fqc2f3fs', '6adwqr0m', 'qf07yssg', '66xk0qqq', 'o7synpy2', 's0oj6auf', '7hrwh6cl', 'skg6jn8s', 'm8ksbjz6', 'rv7vjdpg', 'ls41ripp', '64u7m9a3', 's81k2eze', '8iyvvc7g', 'mpykzno6', 'f4q05w5d', 'jowjkd12', 'pk4x806q', 'mbbvs1x9', '6d3z1c27', '4qqcw50r', 'gb03fb5t', '9e93qzlw', 'f93jma3d', 'yrnn8xs3', 'dzbibymz', 'bek92zs9', 'zc6hnnkp', 'ks69hgp2', 'bruu7mh0', 'u1ll375a', '3s6lce0a', 'vybruu3o', 'unuo41uz', 'k4p1r30p', 'los6q1k4', '265try9i', 'xqopl0nc', 'a8e7r80d', '1c7h6aix', 'rt7dsq6d', '9e4txeck', '694wrmg2', 'v92jk1wn', 'x79oepz4', 'pe7nepru', 'umc2t7du', 'vmqr7p4t', 'n3k5mndu', 't7uh4uba', '3tcnw1pv', 'sybfbev7', 'eprjumc8', '1c0i5q74', '57sy40ng', '4jlhu9an', 'lmfm39wf', 'fxkpfzc7', 'v6f05vde', '02say5f1', '0797wh7t', 'j9wznsy3', 'rndwktl5', 't2np6qa5', 'bydh1ws1', 'zoahecpd', '78fbxcdf', 'xvsxx9yf', '7y38o2jq', '15ow3n9z', 'gzgqjgku', 'a980qiu9', 't5owxikr', 'kfuu2x9p', '3qspd4hl', 'hkb3cyl6', '1bjdtndf', 'd61l4o7u', 'ppe8br4z', 'vyo3w2a4', 'v88bq4bx', '1gqmefvr', '6wttnfsl', 'c5fn8t7f', 'zuh71yz3', 'c3yxk7mx', 'hzs3rva8', '33xbpklr', 'jmgam6sy', '3mv8pkwu', 'h632ll6u', 'jq98jg1c', 'ow1h9o9m', '96o8z5m7', '6tcaij4g', 's6wmq3mz', 'elthb8v2', 'iboz8zrj', '6aiaz013', 'u3baorbo', '2po918ec', '613m5bvb', 'wvxybca4', 'av02xc85', '3vntjg8d', 'bisheysk', 'rriyeqrm', '8oggtwia', 'yvgf606v', 'x075cb6r', '5jyxvgum', '92u8o9xh', 'wnqrowur', '440kruco', 'geuaq1ia', 'fihq5gnb', '3b9u9xpx', 'xlxrjme2', '8diisqag', 'r7h95rw3', 'sphk023v', '1z27qsym', '0x77k8es', 'wlaro17w', 'q6le3ld4', '9cbiudk5', '1sr9ploa', '8h3cbhdd', '6nlpmsxj', '1bx8bafv', 'ii95safe', 'u6vihq3u', 'gf4o3n3z', '39ywzw6a', 'rsewj2mv', 'ky92p4e2', '95qdfjig', '618p3gln', 'biycskxa', 'j25isz8f', '262w8d41', 'sxhg96mp', 'dx2u8o0m', 'b3ze0726', 'npkf3eat', '1kjgzvbi', 't8912ug1', '1dox1kgy', 'qqn9r6w2', 'jgi4erls', 'if4nuhgz', '66gse3jh', 'goyiufli', 'eljkuann', '604bxxbh', '22xb39nq', '8mnfj86k', '21y9rdnk', '0c0i3dbq', 'xr3nrzj4', '73kgcv1k', 'se9swgvi', 'fjcshm6b', 'grhdmm74', 'pmwwq3gs', 'mtth9f08', 'ozdwkkqn', '1mjzzi1u', 'm3fp9q3p', 'ftvysqxx', 'pf8gf744', '6jzadr1z', 'rjz0s2ut', 'slapc5xt', 'ylr2b8ck', 'scptrala', 'iwc8nqw7', '3qpptp62', '6danlh8h', 'lja900b5', 'u4veyaat', '3ngrtmb0', '70szc1va', '484v67ds', 'p2pxlcf5', '9ekxkpll', 'g8saag2o', 'rmvfca1a', 'a5fbac8b', 'a85gf3q2', 'suygiew5', 'epxd03pt', 'mens8hhj', 'yl1h9h8q', 'scgltlqf', 'o4ofqk5e', 'mxgx74ws', '2mvcyvbl', 'yqt131g3', 'b4ujldmq', '789jswjx', '703febui', '5wefp31y', 'mjof5d7x', 'cwhq5wph', 'wxx678gr', 'nmd81sue', 'fex8sd1t', 'wnfma5a8', '05kcygis', '9b3uvsrl', 'dcb237an', 'x6hln4wk', '83uv8eth', 'v9xnst5a', 's5ioyq7h', 'yu7w0t9r', 'ixa3vill', 'da2v1smw', 'rf2zu9cc', 'ce54jq3j', '95o8yd7h', 'awl8p8ce', '122hvg26', '4z6yz8r4', '5ew9vze1', 'lybf3cwa', 'wnpzbhw9', '3uvuo4sf', 'pkq87hkj', 'd724689i', 'e6mj2o9p', '9gtnsyts', 'lysvg3vw', 'jnjrs1tn', '77fss4lw', 'fa1niz51', 'tjy6clgr', 'he54lfha', 'uu5gbhqf', '4xi6s74x', 'i4a6nf7l', 'iklayx0g', 'hchzc4zp', 'vcxozr86', 'mjx9yrbj', '7hp3t53o', 'ne3w7cxq', '7ltfin9c', '7b2mt1a8', 'dw2wzukw', 'o5ay7w32', 'u3y00xqd', 'i9ykx1vz', 'z603fpea', 'z6gwu6vw', '5ipf7h26', '5bj2fn7g', 'epmhpic4', 'tfwsk6ta', 'bqrxnzl7', '120654u4', 'za487ogq', 'mqm0jvkj', 'n5exo3uk', '0s1vgu85', '62gbvlsh', 'rluv7435', 'ybgh9lzz', '1qn5y4gc', 'go48sux5', 'oiro07zf', 'rjy2pb0e', 'x37avz1v', 'xam16kkb', 'xadi97w2', '7mddw7mi', 'cryp0a6n', 'soah9v8r', 'jbi3ldc2', 'o0jqdc7f', 'lmfwvq65', '669od6tr', 'n006c38n', 'd1peqsgy', 'iym3madr', '1mv4pwnw', 'yl6rog5y', 'hlhhvmtu', 'gx69t240', '4jocnfz2', '2n6pakm9', 'gbtj2qb3', '4cqqnkan', 'o9ii9fj3', 'crr9vlq9', 'x5d8q39r', 'swf4bnzs', 'yrgsed69', '1fz90e3k', 'nzjwhpa0', 'v96s6tny', '4ogrl1bp', 'n6os9m5t', 'ogk35qlp', '29mmwi6z', '4e9kavus', '97tkyv3w', 'itpbccqm', 't8oz55wa', 'fxwravjg', 'ar8vxv6m', '1a1ryj8b', 'r4klut5c', 'qpbkl6lk', '2vxnmiyj', 'k1ex9fes', 'dlsj9kta', 'vhvngxzm', '8tqdw8jk', 'ktjqs23a', 'ze73zdxr', 'tqlgysio', '9qtbyx2o', 'os54sk1r', 'iuh6yzz3', 'b2e4d2qt', 'ixoxk5qx', 'c0vn3m34', '41yctlqn', 'r45b4imo', 'locm8rzf', 'ysgrojdx', '108k7gng', 'w6rzvl5p', 'rcz0qw0d', 'rt021wsf', 'vq3budip', '7p47h7e3', '2ntfeoqs', 'dw6cjon8', 'ylgozb4t', 'kh4y44uy', 've51wy5p', 'eo8bpb9a', 'kt1wvn1t', '4dqicuz0', 'kv3qc08u', '4zvs7f6e', 'quuju9h4', '4a7mp2of', 'ivnl3g8b', 'hqwqm1u3', 'b8iyyymr', 'a0drmmf7', 'vm96zvz8', 'pf799wbw', '98l58ini', 'rmmmhrjy', 'lgunckur', 'purbi7cc', 's842zn9l', '1lilrqts', 'eg10tgyn', 'qhy1jrum', '7emgmyus', 'r5hw4l46', 'qi8x5yaq', 'eyxkldmp', 'h8omxukz', '2c381u4v', '1109fcvc', 'b2dsjrv9', 'zh6v1f95', 'ycyww1b9', 'l4ht74rr', 'p6nkl4vl', 't1857nal', 'eu4jkf33', '38bg14zc', '1oth8r7g', '2rx84imv', 'gduhterq', '2c7p56s1', '2ilk2uqz', 'unh10vt8', 'golellkv', 'kx31apc8', '94ljrip7', 'fdzm9xp5', 'ylnc19cn', '03eifdr1', 'juenq67g', '6of99yoz', 'qejmwvst', 's509p30y', 'udlyrbnq', '46us3wzl', '366js2km', 'ufcvecwo', 'sn9fdwck', '1ud5hzlh', 'tss2iaos', '843rf3j2', '94ttp4vx', '1wlganma', 'h6e24h9p', 'oa67njov', 'jnlaywzf', 'xytka5fa', 'bhq4t850', 'bpxlvpz8', 'xo7q4dqr', 'cnzai57z', '03pq9vom', 'sq2pphh0', 'eja5z6hr', 'hlhf39a8', '3bpc5ts9', 'nqyld6a3', 'ozhueiru', 'yyk6af7u', 'ygoxvll3', '4u0appfs', '9950tm0n', 'qz24al6j', 'x7fj0fbn', '0p192lmv', 'mb6y20jf', 'v1rm2fta', 'ma913jle', '7bg4tx9q', '2eqbcgym', 'p2pnr8mf', 'p760z9jo', 'ykqx065h', '2dp78wt4', '76gn1wnr', 'bzyww901', 'v10tfut9', 'h8lm28mv', 'mwwcsag1', 'ev3fpqq0', '8ub3fjwy', '4krbz13g', 'urdi5pzs', '8p6pvjkn', 'ce1us64t', 'xs3ekg18', '60qw00zc', 'k31gi0q6', 'uaamv1xx', 'q858v1ay', 'fa5r7axh', 'm28v8b83', 'c58qowjw', 'a9yq7cf0', '127c5bve', 'tuj416jm', 'p8b4wxb5', '010xjb35', '4g80mq3g', 'iiulyw0g', '2fg2jaix', '8iclmzfc', '8j41fvyk', 'wro1w5y2', '0anag9ax', '7uk2ialw', 'svwtfshi', 'a7w3el5w', 'dt3li3bk', 'sbds3xrx', 'pdiyqdb3', 'qyu3234g', 'y01j33dz', 'dubykt2l', 'edga753o', '3hiqe4ce', '10b5s81p', 'imm7y33t', '87uhx3pt', 'rjdb1qrt', '6q7bi1mf', 'zbb0fjv9', 'c7uab2ll', 'm9chmnzc', '6hyrkxdj', 'ew1cw5u7', 'dkwbkq1s', 'xsly2dgy', 'ssbh76pv', 'vyd724v4', 'm8x5r6yi', '4cgjnfue', '9cr1in4n', 'lj7x5qdd', 'iyf7dxqc', 'c78paqbn', '1nwns3b9', 'w66gtl3b', '7pzeu5d6', 'bksp9brp', 'wztxnvkv', '4tnl6boh', 'yx3j6373', '6tjap6om', 'ra6bp6fb', 'n259n9jy', '24io736v', 'ibf0s0o5', '5fyihr20', 'myvhzny9', 'ydlbvrog', '7g3keqx9', 'omkikfs1', 'x4oitcyh', 'cikmkfpe', 'u2r55rtx', 'h3s156x1', '908mbz6a', 'qbhzdjvf', 'p67xqlpt', '7y0yecm3', 'kwy9puxo', '6p8vvscd', 'dmjp2faf', '6ytoh0xm', '3dwp68yf', '0c1vvn2x', '7cpdrulv', 'gj5ax6yv', 'utrco6u9', 'za3qypgg', 'm909sfe1', '01v2ittu', '70k0cvy7', 'x7qqzpmw', 'xmjyaktl', 'm04zr0ls', 'hy1xrw6q', '2dkht0to', 'se7gdpna', '7pk2mb7m', 'apsh41d8', 'v6acv7zg', 'lzy4cc8s', 'bhlnpi77', 'vuqq8rhs', '785hnh19', 'e2vqvsbt', '3cipvyf4', 'p3uyilzg', 'ydw2n08q', 'vmhrt6bk', '1z6ubp8d', 'f64et6vz', '8tygt4zu', 'le28oadk', 's9jy9f0v', 'puwoy0pc', 'qoedf2vj', 'xcw98ktg', 'vkbukjj5', 'pl3qlcpo', 'rfxldn17', 'ju8kxm7u', '5diml4t3', 't4tg0o8o', 'd3882bja', 'io1t141z', 'ds3y7m74', 'kjkj0ohp', 'o9icinur', 'zra8fixl', '34uorjgw', 'y0eb7nc0', 'hgv36fdh', 'bwqfunit', 'ijlvgcxp', 'lzy268ze', 'fr0n9dkg', '979hxuw0', '1puzasej', 'umh0qvfu', 'b9unqnkb', 'jp7cyxu6', '17hdeiel', 'awb1stsr', 'k7cobfw8', '85yh9f22', 'hq8g1dl5', '4y5o2p44', 'wu56bulj', 'm1evoj5p', 'w574amkw', 'dhnn1oni', 'izzwh3k2', 'vwvgcw1t', '1n7dvsjv', 'iomo213v', 'xlgvsjje', 'mgfgino6', 'b97rmrny', 'b554zin6', 'mc7mnli4', '80tbv0be', 'shf6cr95', 'ou68j2p3', 'odl1nv4i', 'd6867elf', '0lzi15oi', 'xeq8x5he', 'kpfgg3nf', 'fmh0rj3h', 'dn6zxvp9', 'qr18kldl', 'zs2h3ytz', '1hyhcne6', 'w9mij6c6', 'unpcyp5w', '8kubvl2v', 'ltfga11n', 'a8xdb5ho', 'b4sbxeyq', 'vcn9xq0t', '8svo2w3u', 'todb1d4x', 'nmx91h0a', 'ks7uijjo', 'yrlol4pw', 'm5jfd3ym', '2ijpb58u', 'pvrf5z91', 'ns66mpou', 'rlszm1y1', 'cb0mxb2r', '4fioyrgh', '9cvvxy4d', 'yi2m5ukw', '0fq2ngcb', 'hosaskty', 'hiubm4ao', '012wzr32', 'm4y1hyf0', 'drxrwpnv', 'o6xrzvjx', 'ldtrtjqu', 's1az6o5c', 'f4c3r87j', 'g6u8d4xo', 's0fd5b52', 'ejzy0jcn', '80ls025w', 'yr0oxj1r', 'owh347x8', '9ql29qd5', '00wpgq34', 'm7raqky0', '8yoxgmeb', 'zb3p3ae8', 'g68xlu51', 'vf2pw4ki', 'wnd0ewz1', '6g34qwer', '0jw1p1b3', '2sdirmun', 'ml2jtvbt', '3n4tjvhn', 'o2vmzb3w', 'x0577pkd', '0cnjtxyq', 'ghb55qkr', '6vln3erl', 'o8uadujh', 'b6no0veb', 'mmnfyz6y', '8egln9pr', '4yv8isaz', 'qfckfxhk', '4cixlvbc', '8jf3fnqe', 'sbs9zaol', 'h04lnb8y', 'j3zp1a72', 'crno9sy0', 'f8fpn0qz', 'eccef0ye', '9217t0ui', 'ny2mw99z', '5zgtflf9', 'hnxmtbrf', 'vd2d5xhq', 'iz6q2633', 'wwjluneb', '5r0u2mmo', 'zc3nisvw', 'cetdqgff', 'kfawrgzg', 'ovh6u0fm', 'n6juf8tw', 'fowtr5wx', '2j2jbx17', 'af6v4ed9', 'jfp1tq79', '24vsanox', 'jim9gr7v', 'bdq5mxv7', 'lisp9nd2', 'aujkqqem', 'aoor3ofw', '2zk2vcck', 'qxvtmdbk', '1915kvwk', 'v71tbpp5', 'jx1ws18b', 'u634tgwf', 'd41avrz3', 'xxsk6dnp', 's3u60wim', 'c6w7eucp', 'suo4h19v', '6ityxc64', '1rv779pm', 'ckdlxo0t', '8qp8o2g2', 'mpcb1mwl', 'ly72scru', 'pasyk1c1', 'pvus33s7', '6up27wou', 'cap2rgf7', 'weov3jvz', 'igobg353', 's92chxzv', 'lh3txzx7', 'vn3p2cjk', 'c7tf93vm', 's8lj0fys', 'kcklocnc', 'dthd9ecw', 'x0l4hrcy', 't8v8lqgq', '2cn2b2a4', 'mrsya6wz', 'q6oudeso', 'py3ql8i9', 'msethhsr', '08xcud0i', '9jb3w0zu', 'ym2bkni8', 'jig71kpa', 'jmy02i7z', 'oj9lqd6t', 'hek2s2nn', 'avl980hg', '2nlnyxct', 'wgnddxqp', '0cbjg2dc', '6b4j15oc', 'y6iuwq9r', '4gbmp393', 'cszor0pd', 'krqe4jfy', 'jtamxl3c', 'zhw8vh3e', 'nw1erpgp', 'rroaast3', 'fkisgm2j', 'zpsuk0dv', '7rh07bii', 'neinkkzm', '75c4w8gn', 'kkrozq9p', '0eq8ms7k', '96wi5b3a', 'm1cmykp3', 'zfm6xjkh', '37kll85l', 'ntovppf8', 'y5ziurm6', '9741tila', '6ckw729g', '09kvh4bv', '4pij0x9q', 'afhw0450', 'wjlf6rdr', '5y5w20n7', '1mcmwe7f', 'vx55fovr', 'wa6jic6r', 'upf28n5q', 'hkm8yspk', 'vh6dpf4e', 'jp0hvcg6', 'aqvn3gey', 'dmt5j061', '9v0lsko1', '1mjl7f8w', 'ef1qr82d', 'hdkmzdq9', 'u7dapeuu', '09acb1kl', 'l6qjy633', '35tx08hd', 'e1t3qcw7', 'mqujbiu1', 'fc46k594', 'bm8z2yji', 'ugmrg6jq', 'auj3gib8', 'anw74fh8', 'e2pjtjn8', 'qac15c72', 'var73tav', 'u0bw3uj6', '1ip4a05u', 'mczxhd73', 'el1bbx52', 'snpjf9p0', 'm5glyy6p', 'jbrqo1e0', 'adz819gj', '6tz59h8b', '74sr4yce', 'e1iek3jx', '6l79gzeh', '94k4c05p', '3t5gf66c', 'zj3ftw1e', 'f6yrg6o8', 'n22oerlx', 'xzxefda5', 'cmd8k1yi', '3v654s4q', 'mhbtxriz', 'z61b6j0z', 'roxa38p1', 'aokaaxhy', 'cnrs0zpg', 'sz8ieltb', 'uydqogys', 'yryz5lf4', 'ek1m4yiy', '2ledrw9j', 'pf6xk34i', '59lxpy1i', '6kddn1la', 'uo29rlxt', 'h6djco78', 'pwul6lco', 'zk0l7km6', '06d8481f', 'bn8qjliy', '64fuqj3o', '5el5urni', 'a18lw6kf', 'zokdgmme', '62quc0fk', 'cee8i0r6', 'm9ns52fc', 'qlyemeqv', 'uvkq7m84', 'jo7ty7v4', 't7rxmzvi', '8k81jetq', 'pp06m5xk', 'uzatktbf', 'c4yl3wps', 'itjq6jfg', 'z9rksmxy', 'xadhiuhn', 'n6wwg44h', 'zgdvo29n', 'rpnuauwz', '9pxr8w2w', 't5oz4lf2', 'foaceqe3', 'ciihp7pj', 'wjfvr7pa', '5ix0v9pu', 'uaoh7pkk', 'tbvd5nmo', 'gm6otsfu', 'eamhyai5', 'eqw8ywrl', 'tssl7ycx', 'jzqgj8f9', 'txa0u85o', 'ocij1wi4', 'nugixx3e', '4pbo16mi', 'bmtd5jak', '2z664ffl', 'b62z2fsg', 'sgvbfr1u', '9fq99fil', '0vpmazri', 'tb4k0rxk', 'fwnggl65', 'x706cwwp', 'fblvgjk5', 'h8zqis1p', 'yb5mn587', '2ayni3nk', 'bgdemah4', 'us6tl3mm', 'uf1ovjpp', 'kiik5k3x', 'yphe6gc3', 'q3z2ggib', '15vimfk9', '0mmlymw8', 'u59iul28', 'mbza48b6', 'zznevkkr', 'sfjmqeju', 'u33qwt2n', 'ceuf0zep', 'xv3wx4v3', 'q2ivrcya', '7js2dlxi', '6nhiesh9', 'r0w0jbpg', '5cdk0iap', '5szalmsz', '95sck64d', 'igtwn6v6', '9ybvill0', 'auum3tn7', 'dz9mlbqv', '9y7ebe8v', 'lrb8imsi', 'f2rv6chp', 'y5wbl6uo', 'l5d38wst', 'akp03v8x', 'rtirduja', '54l4p10o', '8hjku1i5', 'wifbpmei', 'budhtlxv', '464qi674', 'bvdkxire', 'qk76szs0', 'iinewvxo', 'ed26chns', 'elotmmf4', '9luk0qtg', 'baqg05cx', 'jv5wyup8', 'tcxbn126', 'jg3eivsh', 'pu5548gq', 'gnfyn930', 'b8vtmr9h', 'wrl90t6e', 'msga2t32', '71n58esp', '8ttd3i1t', '42luhd4s', '76vexhxq', 'nay6x9y1', '3hiki0fx', 'zdjzmy51', '0zaesj7l', 'u5pe6s44', 'cnzhfuhi', 'bvqf6qi8', 'a2xscegh', '823fd3mv', 'n1yf0o2o', '75anahxj', 'bdu5kbsv', 'hg6ago7x', 'arwb5wr5', 'pde3w08s', 't3disym7', 'kx8kmdej', 'tvlqmytl', '4hlcxccy', 'l3jk92u0', 'mqk85l7p', 'z1nt8ktx', 'j2lanfjm', 'ssj9w973', 'lyh7iz8u', '5awga6ur', 'ulytp5ah', '9kw3j90r', 'ul8wjrmy', '20s6105q', 'bjl1yace', '0cbkatcs', '3x1tkxw7', '019lkt6k', 'wa7sadvu', 'wbopygij', 'sn265wgz', 'lr8kfa8a', 't3x01cf0', 'gbtyjkws', 'yvy532ml', 'jpjope81', 'm005jyta', 'wqc4o648', 'o5hx0hfu', 'pjhl2o13', '2bjpekxv', 'rdfyrbud', '1rzygomn', '8sl534n9', '2hfjg8zj', '9jvyjc4t', '2dw318eg', 'bskzaxs5', '4b3i30hi', '0oa4x05s', 'rrer030y', '6ul6057r', 'hhqkibpv', '6mij85lt', 'b1e0bbrp', 'b6to1v4u', 'tpnrmj4r', 'l5oaxugi', '7nwe0fkk', 'c64bobhy', 'pj2u2b3a', 'wqt8wv17', 'sbz3uk5h', 'hdkpzsmm', '140ggbhs', 'yugdgj2a', 'p0tdc5zz', 'bdcjw336', '66a0sj6g', '55fyfdd2', 'qfn0zues', '3djveo4d', '5nu4mbax', 'wz2b6b06', 'rpzlgkki', '1zev83lc', 'btuoin6d', 'hewibmib', 'jtu59u99', 's6qw8l4e', 's73si6ah', 'e6nt7xbs', '4rpv87kl', 'zndxj6d5', '5qnxonie', '1lke8uzi', 'vg8qtk1w', 'h6jbeh1w', 'zx3he4h1', '8xnfwho6', 'dlphvji8', 'c7g41uyz', 'snfual5n', '3710ddej', 'ls147m43', 'ycmmg7d8', '5yhe786e', '0o9ru8zz', 'x57r4oko', 'dm2msjpl', '9dhldjjp', 'gp6gz0bw', 'v9co078y', 'eb2guuz4', 'm6j2khtp', '5iqvzfmt', 'da7z762v', 'ieta7i9o', 'ud8gevcc', 'ayxfsx1q', 'k1lllg6y', 'awyk81ka', 'x3dergtf', 'dwfij0ad', 'vrbb5a7t', 'j9x6zmkh', 'n2wrmjei', 'yvtmnjhe', 'fysfid5z', '6jac3pue', 'hmd3quxa', 'y5ccnqev', 'ws6b99wt', '0qol4qy4', 'bjlbfp4t', 'mkv1jc7u', '0wy0e7ef', 'tvxpckxo', 'xni218yr', 'xdnop0rn', 'v816gp4h', 'd2a3qmfh', 'l3ss4j55', 'zbgeepxd', 'q9bvij4r', 'aqal6kt8', '7c37elh6', 't3tahuhh', 'vnynbijl', 'tof14jke', 'l477gbvs', '3r60as1g', 'np0vmg8a', 'zqy3cbwp', 'qtuoigi5', 'hr53v3kn', 'bye3ikjv', 'l6w03wht', 'r6jebw6f', '8a1x7wkz', 'lszbnnnl', 'uf794owe', 'bi7z535p', 'iiz7v7st', '044b0aly', 't2k7uov2', 'h5sudi4a', 'gxpwnq9s', 'nsfilmh3', 'oh45ifkx', 'e594fbfy', '8np6tk82', '8k1fmlsl', 'wtthe00m', 'x5at2izj', 'aazhb87j', 'srizph65', 'mvuvpxj7', 'swvmut1y', '6vp8bvmp', 'gsbumgot', 'dyju6w15', '7qxh3art', '8v3joy4l', 'es7q6c90', 'xobe5ikz', 'uyoqme05', 'ura61ma2', 'bqo1wf0f', 'ivwp2zqo', 'k8zb7ecm', 'hluzni99', 'ma4sj149', 'sedo8a21', 'be10yf6w', 'dcwlvt6t', 'yzj3zfmu', '4jrro0gd', 'l405c850', '56z95uca', 'fwlqkr7f', 'ql6bdnkw', '9mjwwht8', 'wpjb8ay4', 'pn4ctd3t', 'oyhpwut7', 'a4m01fm3', 'cdctlduc', 'jlhykfbf', 'vvw41yvn', '6mv46jrx', 'cbs3h8qd', 'va50jy5b', 'mknutfmk', '8o6vusrn', 'g9fnq1rk', 'qdcnkpve', 'x5atjxcu', 'pz9vp7cv', 'qjbywqbs', 'f0tkhi4f', 'ahceohgl', 'ywbnfb2j', 'mxqg602l', 'rf0f3p4z', 'k0s0dadf', '0o6agnrc', 'ozgmbfm6', 'gptqp18l', 'th3wf8zz', 'tk1bxztw', 'o2mojn2u', '9nkq1g18', 'ntgxf4ig', '3c2i7xrx', 'nrmj1qre', '7zf5tx94', 'lcrbs2op', '6sq1d5jt', 'll7kv6yh', 'b7yhjsq4', 'eckx8o69', 'q8tz529g', 'dghhp7xh', '7h4402uq', 'klb383bu', 'qli7nb2t', '6dr8xy8n', 'wnn7583h', 'p3pq67in', '0pkzj7oi', 'lgih421l', 'dkeo6nkw', 'p38ondmw', '95h3s7he', 'xv9qr1m8', 'tg69kfof', '982ufh4z', '6u9cpkmt', 'inso1d03', 'fche4c7u', '8g5fk0ua', 'o7gd0ubb', 'ol1tz5r6', '0pb0m2ak', 'a70eedna', '7yortrpn', '23847ggw', '1d5zbdrh', 'stz1rgrp', 'e4q8gtmj', 'yi1lvevy', 'tny4bn01', '3k3faofl', '28ppx6qz', 'clpkd3qu', 'szkiilb1', '6qmyksrh', '551j6f20', 'ourqalwe', 'nnr5e8eu', 'zm4cnipm', 'prhkhnyt', '1b774vki', 'r4qcpa5h', 'fwnndl5h', 'in044snd', '44vci8tb', 'a9hcnea4', 'pfzwsyq1', 'z5q0epfx', '9zsd3kvf', 'yuomsrss', '8fbwiqqj', 'gxv4vw67', 'zss38mct', 'twf9hbiy', 'dxff9l5i', '33fmcxdn', '6cyza1s7', 'jmgh83wz', '0aj2tckq', '3ora6css', 'e4aha2i8', 'fz3er5ex', 'z7cpoqsb', 'vdv4a19r', 'ane3dr9h', 'kmkgxgq4', 'yxjg42um', 'dyg2baog', 'i8da3sgf', '7wunz15j', 'eie4gqba', 'kamwgcuv', 'yxwvchvu', '0rs6pbly', 'sz8dvkg9', 'tcl5lrjp', 'tl2i45k1', 's1slcbpa', 'zzrakyw2', 'bdubr2w6', 'pu98wc57', 'eizypepg', 'bldkq7y6', 'djzevyuz', 'rlj4kme6', 'fan0sf48', 'i8fjwzm6', 'eu0qns25', 'jp8y1owf', 'hsp5goxp', '3eue82br', 'rpnpzm86', 'jh65bga7', 'abatx2gc', 'hkpi91bz', 'gwi3t8mu', '5sctqkwr', 'xr7wo8jx', 'rr4yep41', 'q1gg12zl', '4mpryaor', '8jutg4dx', 'p5ohjzhs', 'ogg4tqkv', 'mpkokg23', '02ukk42h', 'zf77xmx9', 'um0gwm18', '0xd52xx9', 'yy745jhg', 'q5xp7pa1', '0jkl2f66', 'k85w7fdb', 'yk3fp4qo', '5hlooiwd', '5zz2k8w0', 'lwgkk4k7', 'smrlwpe2', 'xb480fly', 'qvklgp5t', 't7eb91w4', 'ce3b5rhi', 'bstnt96p', 'vdkcxbkr', 'elaqvw2d', 'd7ks2m4u', 'fqknallb', 'rxk50ayb', 'nosiiye2', 'vb2v6928', '5kavwv3u', '30ojo2i3', 'snmkitwe', 'f43jek0j', '5ijqhmie', 'u4kv0ixf', 'tex4lbpg', '4jy1wl6h', '00wwa747', 'jnwinunh', '8kko2kq1', 'j8isev90', '5qybnq6p', 'erh8ggh3', 'pf45ymam', '6s2juliu', '4yo27web', '1b04yzah', '4mea2yo8', 'wdxfw4s8', 'v4sse0rr', 'sbs9umml', 'j4yjdjjy', 'uunvhmux', 'icwzey8o', 'kzcld0b3', '3z5wh4ne', 'sxz3r0i8', 'j0d6jtaz', '857qqfxr', 'zpf2ri9k', 'srisqld4', 'g6vuczg7', 'tifywvrz', 'n5sc9l5d', 'fkea2rp8', 'be1qnr46', 'fmyl3jgd', '98pbhxae', 'fayb0of2', '6vuzhu9v', 'eovluaw0', '9myiiuzd', 'iasyn0oq', '54gf05lj', 'yacf8jjw', 'upzmeeya', '4y05t72c', 'thoay0ia', 'vnngxbh1', '4bwatc2r', '4eduslpb', 'srfo7bvo', 'vptva2n0', 'hkuschqe', 'u7wxfv6l', '3i107kqx', '7jeexos8', 'gqkv2xqf', '8haxy0qk', 'yp2o5qy5', 'q1qmkp4a', 'sy6fld5x', 'g1i6buvs', 'tluwpcys', '0i418jsq', 'sau0g5i6', 'ebx7ixrt', 'mt29ackq', 'x6a6abzh', 'e0sqppeq', 'lrpdm74j', 'hkfkugzn', 'e58jd3ac', 'v1hbust8', 'fo9hkhhw', 'cpidk3nj', 'rszunnv0', '5kfg7pxg', 'boys7vbd', '5cv2y8po', 'tt5jmq0y', 'ow2td26s', '4yardtss', 'a9jprzqg', 'zxrrzylp', 'hgqclv04', 'yawwtyxg', 'xabnwsvo', 'a3i01y7v', '5zwrn71f', '8tjnrp0w', 'onyc1l42', 'z02wmrjh', '4wpikzk0', '3ajfb18m', '50jcb92h', 'l0g04qgr', 'gvu6vjz1', 'o5dxclmm', 'c91mthxv', 'vwgmzehs', 'y8egpokv', 'ds18bvzq', 'zd7fjp38', 'ronxy37z', 'jtcixc3n', 'i8tacj63', 'e6ooqmga', 'ps0wc4q2', 'nbhdyst6', 'oixvk7mx', 'kp2ewbn1', '0i2i813e', 'nmu49xew', 'aelha50x', '399ixbrn', 'oo57lyrl', 'vc0dstmc', 'vhtr3l0w', 'ievtvhge', 'gmsd6030', 'oi0dq0im', '606hxe5w', 'iqjm27yv', 'id7cym7b', 'tiutz77j', 'in6jfpwy', 'fa5qgum1', '8383qbib', 'ezgrfdhb', 'vpes6y4o', 'jwyudahh', '0hr744mg', 'tdo3t60q', 'i9mqeivq', '4fvqvnlt', 'k3ckqtob', 'io1dcv00', 'nytwp4rj', 'c7fdfou9', '2juia10z', 'fx7c22fj', '2zsl30yu', 'grctboqk', 'g3s4iavk', '646x4ygm', 'ieededv4', '4bxnm91v', 'p30wz7b7', 'ebpmrqli', 'ro8ed9nm', 'drgrr23k', '8wjx7xlp', 't3gr6bc8', 'vq72zw0n', 'dza80e8o', '6j29kyoc', 'shqk68ve', 'ure468da', 'dvzsf5dd', 'gkfur9ym', 'xgtyhvr6', 'lhwvefh0', 'wjx18sh1', 'ufu9ggrv', 'a3g0kdfo', '6j2peqc1', '0noi24az', 'hl1tyc1f', '12u18uww', 'f76ypfma', 'vrzsseiv', '61ayjej8', '1t9wgprm', 'xrntg80a', 'ishxfinx', 'd3x0rekw', 'brkfgbbg', 'wdsqzjs0', 'ib6x8wbm', 'vyieerms', 'j1xqfwy8', 'asouru6u', 'c3cu4dnt', 'vtfj8vtu', 'tscdpmqc', '88vo9l4v', 'a32i5kie', 'ltqt3n95', 'mnfvb4us', '4jmgp3q1', 'guadaq1d', 'zdw0d4e7', 'oqn31x4o', '6rtuwgn8', 'dt115y7c', 'dphrrtzw', '4819g00y', 'q0ap724t', 'uqot1ed6', 'pbin8pn1', 'zbp8a8dr', 'uurwwesc', 'q41plzqq', 'ocpu7bd5', 'h5gp0tb4', 'lfo3vdy7', 'kgtou9ca', '7lkmq792', '7v6wfye4', 'q5ri12t4', 'bmx6b3su', 'j2lw3urj', 'ze6nvje9', '65obz0pl', 'h69je8gr', 'ypekmm3f', 'hmpf4wnr', 'wmid79yc', 'anqswymg', 'p8w61scy', 'w43wec48', '09dv2tak', 'x2gr044r', 's20y6d89', 'wd5nxnau', 'k9seqoit', 'hjy1z1kr', '0nk88tjt', 'v4xj7fti', 'euaq79jc', '00w12i2w', 'ip4bxki8', 'cjkpr7re', '2i0h4qbf', 'qw88g00f', 'zzvetjrt', 'r1e88t3g', '5307knm5', 'xsadunsd', '1b9v6q4x', 'e09ezzke', 'rkkeb1vp', '01y768ks', 'mfwfibki', '8ru35w88', 'oj3tab1k', 'atpcbun7', 'vwcr3x71', 'kfja1i6s', '855h0e1k', '6s3b83av', 'y6ncunak', 'aq539byf', 'wzj2glte', 'gioqkeda', 'l3d19xur', '2txwhonv', 'sp2d5nt4', 'c1zo8wwa', '8w58jh1s', 'aqsfi30i', 'auvtwmcq', 'ck0n6zyx', 't9wjonr5', 'fhqkkcod', 'ozsvrq92', 'ldziw1pj', '9ur6oeim', 'sp542req', 'aoni2yi1', 'hrfzwcok', '8epe3wae', '5p5z35o3', 'r8dygehu', '8avgujw9', 'qfim8nu5', 'zx6utpd4', 'f5rhem04', 'xq97bsbe', '1mijh76e', 'nmsju6q9', 'fu9n7xg2', 'vorh9bjg', '3g7ed7lv', '9npnyexo', 'pn832s49', 'x78ron57', 'lh1wtyew', 'ljjry7gt', 'ewersik9', 'ipcewwfg', '2zk9t7ij', 'xcgs2gm5', 'irvb0kda', '3rbdzdo9', 'onb9v9fk', '5mrjbxgr', 'yefaqud4', 'er3i6799', 'kis6dsh0', 'twfmo1o0', 'er1epm8h', 'ipt90di2', 'orav0h5x', '9pw9191b', 'r8wsztxw', 's33clv03', 'q0w8t5k4', 'so9f8opa', 'lm6rwhki', 'efog9bri', 'vcto7y2t', '2ju8t2cz', 'g24g97uj', 'm0i5ca18', 'ml2zoxzx', 'utm69svg', 'f6modwmv', '0t4c1o8l', '5kt7n9ue', 'vindspai', 'm8m04v06', 'tetl08pe', '0i3136dc', 'ymcgs03n', '1xhzonc8', 'p4gy7dx7', '2lclggn4', 'ofis4cgy', '1affwhg2', 'wv2v1wuh', 'a2a98gdx', '8pgclu1v', 'xcw0kxjg', 'wjzwims9', '73mbmecr', 'g5bnx0hk', '7cxp64do', 'g01wmmbx', 'nxst9poy', 'v0xp9pdo', '227jaziy', 'p355gqh8', 'ixj4t0o1', 'h2ncyfa5', 'kc17jcdf', 'i3mzzxwq', 'wd0y67g4', 'mycodht1', 'uc7zt4yj', 'iuubxt32', 'tgkq407a', '0xfp42n7', 'd3dkbvio', 'ngm1w4zs', 'kp5se2me', '3r8jbhhq', '91aug79x', 'ixem36s5', 'gwk4cvbk', '0rayuem4', 'mswi4xac', 'qaurriwg', 't0pm3tmo', 'g7tzcnxb', 'l24oe2pd', 'un2hby1e', 'hrl3dijf', 'jt1ux0va', '3i2s2fg9', 'stbpffvu', 'l9yjby63', 'uphs5gvl', '7scpuf3o', 'xo9yn0us', '5cs56jbb', 'jz4dwnt8', '60mzkevd', 'rxdw553c', '90t1pkth', 'o8ioibdl', 'qoz22q3i', 'ge3i42xm', 'q5faxakm', '5neboyyu', 'vsr4o87a', 'wg979q2d', 'z8suucj7', 'itcat288', 'bk0cohrk', 'fwq5we2k', 'h0651d5m', 'v2tfykzv', 'coashuq0', 'hlfjd8eu', 'ztw9gn9v', '1hhfwtmr', 'cxxxo5zk', 'zq51hpjs', 's9w0f82b', '7668ronq', 'mos9uone', 'nwp4fxcn', 'g69rocp1', '5ndo4d0d', '7ahoiaxs', 'tvdltbnn', 'ovuofl9q', 'k232ldz1', 'jxx5k31u', 'wucqt5ay', 'fj8g1h5u', 'gti5lq4w', 'mnn8cej9', '0c5p8sjk', 'fd167sfo', 'bh1sdi2j', '7dsyn55o', '0533r1yj', '1qs60yri', 'bnf9eq5u', 'wuwotxkz', 'b3gowdd7', 'kntb97k4', 'oj6q71da', '50nqjt5z', 'wyh7t6rr', 'izsmxy00', '7mlf7qdc', 'bujltzlv', 'bcg11tob', '1g2mup0k', '5rh06vej', 'kwnmfdam', '71eje02q', '4t6x32cz', '10ck2bfa', 'h4ccfo50', '786p8tpx', 'zcbyhsf1', '8avgxkir', '10crghj1', 'y58f2bp0', '4pztbu51', '813mi5o4', 'mlqmemlm', 'c1023hmj', 'hqhlillh', 'zv382pr6', 'tt4fr1v5', 'ywwmeycl', 'bpvjr7jx', 'j9l9l4g2', 'sfb1xqoc', 'pe8qxorf', 'brpb48zw', 'mqb7f2z3', 'a7u6vu5g', 'nzz9tt7s', 'e9ts1ech', 'uaj372ag', '23zu4124', 'ht3slpkr', 'qoaytodx', 'oz1xwx3j', 'hjw7027l', '6ly81dbl', 'dfa18wrb', 'jiy4cp4n', 'hxj4z228', 'p9wqevx2', 's7ztwa10', '5r5ghgjf', 'kyd7o4vp', 'hrjnc1ne', 'rrls8vh4', 'rq8r7xkx', 'osi2k388', 'nolgne7o', 'u82iqdxq', '780vp2qs', 'hjl4kzqy', 'mzs0gjrs', 'xd1wcito', 'fe4n3xhc', 'l6bhlxfa', '2mro0dbj', 'pgsykrqx', '69220zqd', 'bqyuugxl', '5ylhufc5', 'ko8owym3', '39yr6k9w', 'k8is48cj', '20ha0m69', 'lq0k8ysz', 'iafhms9c', 'pni1lwhd', '4g2p4rcd', 'qix7dmsa', '6gtgpk2h', 'yyixxmbx', 'e0th8tly', 'm90i5fxm', 'yizobbj3', 'j09x52ib', 'xfa39umz', 'i10q2569', 'ooxb7vjs', 'hy3kj0wn', 'tin7agmn', 'i6vgqsio', 'o6qlywsr', 's5xnqa3i', 'xui0yllx', '2bg7k4pm', '64nq8qk3', 'ny85c4d0', 'u0pncmp0', 'pqmaays5', 'izvs6m5m', '8l3y24zg', '4etml8ec', 'hz3bny5l', 'rdpsxb4n', 'gj9d8p3x', 'e6j718bv', 'qoeznasz', 'tj1qjd41', 'dxsujj3s', 'yykfirgh', '4wyi6kj0', 'lybjyssn', 'tkbqive7', 'ccaiyabf', 'if4ni2po', '6n1xbo7o', '6dug28tm', '8zcr8dl4', '6ge2ihcs', '445hxy35', '26fiv7ov', 'ovmso3qu', 'vskftus1', 'euzvhtax', '7y7x1f3y', 'tog1og37', 'ipoban0p', 'mlzoo8xr', 'aqtep1fm', 'xng4jigx', 'bvnn2efo', '1yd35qf3', 'c8jzexd4', 'nd0epy4p', 'ar3b23zq', 'ba0s1x28', 've8oz8td', 'lanxfl2j', '39b9e6m8', '62e02k8w', 'ujiv0j31', 'cue5lzy9', 'mx2vxivz', '8zyzbl2e', 's8s9doys', 'tgjodbzo', 'cyipbn15', 'clio7417', 'eqgg1rfb', 'tjp33y35', 'uty61t6r', 'rqdaa0x4', 'pxikgilo', 'fz9i597a', '2gej7h1d', 'y4etplhd', '8npyp14x', '2g8iqz4p', '3skfer7i', 'iy2fvkmj', 'iqt8uz2b', '9s49l5r2', 'jasbjx0j', '6njifu1h', 'arp8588c', 'iqcyd1go', '3quns8jt', 'xa3iy187', 'j0496jb7', 'h4t56r26', '89fshhlp', 'g5csouyn', 'dag286gb', 'nj5w6eb1', 'ireranbu', '24cth2u7', '92fc8ob4', '634bzeyh', '2068c7k7', 'om24k5ce', 'qctd40kv', '66wia9n0', 'z6fd8deo', 'bymwqehy', 'sw6niek0', '8ljwpll7', 'qv7touib', 'mi78028i', '44jyy79k', 'dfopl8t3', '9je0bo6h', 'panxp71o', 'yeh6mmze', 'i4dr8dp4', '5q6u14ot', 's2vhjr1b', 'q56vydbl', '84s132iq', '0axms5fp', 'j9lu363n', 'kxpbj9w3', 'ufvq0ngl', 'uqi37wci', '3jayb5ya', 'l874o8d0', 'ty27die9', 'o0fp7v0y', '8fjteypu', 'e2zeh8cl', 'df3w9px6', 'tjhsuz4t', '9ksiyyqe', 'bdtet83d', '8o34ilw1', 'e464idc0', '5i9ouv6r', 'cb70u3bf', 'kvshg68x', 'dbitauwe', 'ccnnnfrh', '89d2gzhy', 'x77yahaj', 'rixh4bhy', 't9cihg4h', '070sjton', 'ej0t613i', 'nehorqhi', '9e4v4rd1', '8rfwdu7m', 'jkuatmre', 'zssy3i4d', 'v3eip7m5', 'nj1wkj9z', 'vtav79b4', 'jqe5l0i9', '7wzbvwok', 'kz3umryq', '07mgibn7', 'v62qd1bn', 'twqr5r2x', '5pvaeuiw', 'opc70nw3', 'lc0hd52w', 'sm122szu', '7lgc07ja', 'g13e5lvb', 'rfbuxsg4', 'biwu8uos', 'no3ctudv', 'pnpaqchs', 'kbl5fmp5', 'gfy2ytg5', 'hr1f3g28', 'o43cfrfh', '6lgfu0dz', 'dovijf0k', 'j6pefgpw', 'ykwigb3v', 'waxfpqy8', 'e67n2zcf', 'x5601om8', 'tt5pv4ya', 'syabnvp9', '1nex7gr3', 'pv4l8apm', 'mmx9un22', '5najee33', 'rxxx21ft', 'hpcpr5x4', '78eg7jev', 'mkegxh7v', 'ab4f0e4e', 'l5704xiq', 'xumw5z79', '7hs2p7kv', '3b5qasdv', 'cmrksxzu', '0rf66afg', '1hv4frdf', 'cnlager0', 'n7h8cpzy', 'nv0hwsic', '5n5yjfsm', 'y3llpf9g', 'r8juw6ck', 'seg0ij0v', 'er70ton6', 'zi5ksp3k', 'uvxqz41k', 'uy2dlamj', 'dg34vy2t', 'jhhvp12g', '4y0vex9m', '8e8w74c6', 'figubzgl', 'i2ldyah6', 'p7sfdota', '62j5m5lh', 'l8bv5t3o', '418pxqvn', 'w7rltm38', '838ou2l8', 'drqlbbdt', 'd3iwlpsr', '23i755va', '7pworv7e', 'qbwpwvwy', 'gasxrcif', 're99v1qt', 'ggsccmpb', 'sds0uzsd', '7onew92g', 'd204aq8o', 'e38itktq', 'pb4of7s9', 'r2lvlw5c', 'if4t2vgu', 'xq70rn1j', '2g2c6f3p', 'lbwjl23m', 'exgz2wms', 'bmqp14s3', 'wikgbxni', 'dd6scp6s', 'cua1b075', 'eqdjo5q5', '5b7m2vq5', '7ubuyotj', '1i1u0a3o', 'c5plhs0i', '8ttk0l73', '2a86qjah', '57av5x3o', '4c6bt3qa', 'owoe2hht', '7m21bgxb', 'r53n945k', '2ua7fwmu', 'grkst89q', '3w0l6gke', 'ceu5gsz5', '0w3crrf9', '7ofra2zg', 'n61ukjmm', 'c12m75gq', 'vojecv3d', '54czyntw', 'eksl7pp8', 'gpai58oe', 'oicbf5l1', 'a6cry7jv', 'eblcf902', '5lbf7w2l', '70x129tt', 'um4uam5z', '18r3otv2', 'b4ie7zpg', 'qka7cu5n', '14e8vbzo', 'j3akzx10', 'wj9zbsjl', 'whoteeu3', 'bdz01kch', 'irtssg9w', 'l9mb1iyv', 'oyle9nsb', '7f69swqy', '018lcuxh', 'b7wgxvxf', 's9j4yv3j', 'niov41xg', 'ypwtg3wj', 'blvn5l98', '6jec1q77', 'uk7qbg6y', 'igezjpw7', 'in3bbso5', 'lyj9tua8', 'cb3u1s7s', 'hg3kgc9l', 'rkmpt9it', 'odseltgg', 'ltdajnyy', 'szqrjfux', 'puzmxu65', '2lbpfaf2', 'utf90081', 'yvss97qo', 'nsvm1wg9', 'y3sb03n0', 'x6d7y5ht', 'zdsgkr2q', 'qhtb67n1', 'tdous0v5', 'g82syk8n', 'jwc7mvls', 'k812c3t9', 'cq7d5b9h', 'z4h3gg47', 'pi8nkuc3', '4gnt3kop', 'xui0l9aa', 'mbfpyyy7', '20kjr25i', 'qzysks4k', '5xa2cgeh', 'eeugb4x7', 'mmq44kwx', '1e7qm4x2', 't00cdztw', '4wmcm5gr', 'g3l80i4v', 'wj42sb10', '1ogbsire', 'vyjk82gf', 'sjhr5uk7', '6x5b9pat', '35yj1m0c', '3vm4tybd', 'fexz6mh0', 'xpe70yxt', 'xerhn7sh', 'obm632oh', '950h9lma', 'ds99ilxh', '34nq3ahm', 'td81eyio', 'c7gol2b2', 'ozbm8vc4', 'glji2ko1', 'emmwsv9p', 'eu7ryw7x', 'vp3o993i', '6ck2feaa', 'gqbfusd7', 'bdkwgo57', 'ytn3ytka', '21krtoa4', 'wc3mr6vt', 'b0djmrnd', 'fydhe7lm', 'd16255kr', 'y05jovdm', 'htdgmwox', '30xxnzrw', 'u8566y7b', 'oh497rz9', 'fvkxvvyv', '3eopffq2', 'ty6kj9xa', 'ituy2xup', 't726wvy7', 'sln6a8be', 'dayrvnx4', 'en7lxkqe', '8e8wt03q', 'j9gpapry', 'b0l8gzke', '063yv6mm', 'qlouqgpc', '1d45uunt', 'hmqfb932', '0b287l6f', 'u43000o5', '373hpkjf', 'hnc4f204', 'oqari64i', 'sxqbpk3i', 'avp6zjit', 'tgcqe5v2', 'd4puzqji', 'bbqhd034', '6zlizqe9', 'verg5dhn', 'uwum4zc9', '2nxwabez', 'f38d8eri', 's6hi1lx6', 'mqf495gl', 'qk20gwf4', 'xg77xy3v', '5wrjg1m4', '4ltj270r', 'c7ipqm45', '2aku9prp', 't1m7dxzc', 'wc9gmuof', '3ue8ze3s', 'e1lfri1a', 'dbebthbu', 'azgpwq55', 'hjyy4mon', 'ih3hldc4', '3ib99gup', 'dvjh2bwp', '155qtzz7', 'fcoqqd8p', '8pqudda9', 'sbkjo51g', 'f518jeri', 'z4yj4kjy', '0z2idch8', '36eawz3k', 'hyc2q8nq', 'j9dd53hc', 'utdpgyex', 'bsyv6s5z', 'opvhwkj5', 'mv7tm7ak', '74b7tzas', '2nrxasfb', 'ohf6kbhn', 'qhf8vtgh', '92l5sm9m', '5qm5svad', '4e4dmb9y', 'nk4h9hwx', 'tgju1ztp', '2l71gyh1', '1pzbnmvo', '5id01qti', '7gdditcp', 'w4w04xd8', '40ucqtql', 'y8dezcjc', 'qi2tx7ac', 'qe6pk2tx', 'vm8usoay', 'u0cni077', 'u6q8l5qe', 'ru2q4lao', 'prlwwk8k', '1go3jjeu', 't7unkhx6', 'l66agptb', '5qyx7jgk', 'uaxmt3a5', '8pj2s9td', '9gavn1sk', 'xz2k98ru', '95574k6h', 'xb1wg3xv', 'mw6k5zmg', '4cuentxm', '3aeqfr4x', '080ofl30', 'lnr4w3ek', 't6qnpy2e', 'p485eoxf', 'v896hrm3', 'ioy27h2s', '48hxl439', '9mzt0fim', 'rcdhji6c', 'ss1r7ljj', 'g8h6cpzr', 'irji8owi', '2xqarg07', 'u39eadvd', 'd9jiip9d', '2swnbblz', 'eml14md3', '41jdx4cd', 'ujqtb0la', 'lbcrmplw', 'e9pwguwm', 'wtqy1dkv', 'nngei4sr', '1iq7vrko', 'ly7q1x7u', 'jfge91uq', 'scg149o9', 'ricxulx7', 'r42dqo1b', 'fezphdlu', 'yjmk5csq', 'hslh3kjr', 'x39uq4gi', 'hq8uvnmp', '241pr7we', 'd7z92ifi', 'v97dczcc', '63zo59sx', 'kuih52py', 'x2okat05', 'k81cb2xp', 'zta2pv7k', '3x0qhjhw', '2pakmsp8', '973h46as', 'suiqh3gv', 'jxmosjto', 'c8rb8yqh', '3x7p11wb', 'p4tsi1vo', 'j9yna0rz', 'shx0i12m', '5766ysbk', 'tuh0v6h3', 'lv7bwsz5', 'gs0r8izc', '8c2n1c7u', 'q9ihm99q', 'i8ap94hw', 'rjyu8lfl', '0usvjfos', 'wajppdvj', 'xz0e4oem', '9k9qx92b', 'dz9lzlqq', '1m0gelu6', 'xxr4p114', 'hb8baoz0', 'uqlam93p', '1bmpvmdd', 'yl0zxt5w', '3ocr7481', 'tx6duxcg', 'ov3bv7qo', 's5n7ecr5', 'no4j9b0u', 'yhlrtik1', 'k21yzukj', '9bn7ewg4', 'toxfm2v8', '2folsh35', '2tlwaf2e', 'll857bgu', 'xzqaovkl', 'syp7pbyq', 'wmxv87jb', '77tcpvg0', 'skvtdicz', 'wg1s8ruj', '0muu5go4', 'g84xlzwe', 'f1xxpg2d', 'xzjzko0a', 'xyszdeuh', 'vz2o7xm7', 'ktfxxk49', '3ogewg7w', 'ja4ptlg6', 'vrfduv5a', '79dn6z3l', 'e523mnz1', 'yqateked', '9s1u3786', 'an3twhzi', 'sxw4l9tt', '1j8z8lak', '0w8is9xm', '897vsyq8', 'yv35pf5p', '3xq5jwof', '2rjt47j9', '6axmz20i', '431pyuzu', '5hsem1x8', 'qvarz07z', 'u57d0of9', 'obdqkbtw', 'rsk29kha', 'uo78tsir', '4c1yux1n', 'ora01g7i', 'eh968rf3', '81i94cl7', 'vzttthvk', '3posyr5n', 'zfv30l3o', 'd599uera', 'dcmz4wcn', 'ermuiujq', 'pil1v1pu', 'k0nzt0tp', '1rghbp72', 'dd2ixk88', '01f29sac', 'wzred0dx', 'iorwgfbj', 'v1xb9uxl', 'pgr0q7st', 'ob6pobh8', 'drpumtct', 'if50cwt0', '3g2j9tnk', 'cpyrnkgu', 'ownfvm9z', 'rtn2tr7t', 'ujrczs8d', 'ige2h4pd', '95vqpads', 'h2tmfp3a', '2m2h1qsh', '9o9dje79', '6csxpitv', 'f7eqdxpv', 'i8o1bv10', '7dfa0iww', 'if9b95q1', 'xsjnli3g', '6hhkhi8o', '7kequq7a', 'w5ek32s0', '92c3shlg', 'xqzdx2zl', 'p7597c09', '9g24xg9x', 'ym420wga', 'svnn4ptw', '2k572ce4', 'alzht0x5', 'vpivq5gy', 'hyezvzl1', '1twgz5ls', '3dag7abo', 'n5ofhqcc', '3oeoh7ol', 'gjkwfkox', '877m7vsb', 'ph15z424', '01uxvjvz', 'll7e0gua', 'vi6ib1fs', 'dvwusxki', 'hs9u94a7', 'hqtm8tmw', 'r28aw0i0', 'yvad1wbh', 'mv8wdbbo', 'j5p5mtiq', '1czjl0hz', 'g042y51w', 'ov6bf1ff', '9dyw9dth', 'otf7mxwp', 'lfe7v7y3', 'vjpk7vb2', 'qx3fez97', '7kfzdvbx', 'o3bryc7u', '9f2q5xxc', '8ohchx8p', 'kp1htgtb', 'ohqo54r6', '6y82f4wk', 'uvk48t07', 'h1tyns3s', 'bfvdhai8', 'o5zyw0ug', 'gkewgn8h', 'tirod4t6', '40yhwoke', 'rir4urz2', 'l937xntf', 'o9lvtrrn', 'fqu5c4nl', '0ucanwx0', 'mb5ilt6l', 'jvvzu4aj', 'belin2jq', 'uu3b529c', '6j3227rs', '2m67ob0f', 'psrijhg2', 'ryzwmngy', '2zr0hsps', 'u9206gfb', 'fv2efcf2', 'hi6adbch', 'c1kwd9d1', 'lf301q9z', 'ux4cflqc', 't9nx7a7b', 'bei8144f', '1hh1vpa4', '77kcjd7d', '9pxrhzog', '9d9mngk1', '5u6jt8p4', 'kwjiyywb', '4gnp66pc', 'e4eej0km', 'uxdferxk', 'c018a9fr', 'rozf6u2q', 'irkq0343', 'p37p9n3x', '2k8lxlqu', '9rme8827', '4i3jz4vd', 'kqm98bdv', 'osdl57i4', '25q3i7nj', 'jng2bgve', 'd7nivx10', '9fp80ujt', 'y8t4l6b4', '6w8qcv1c', 'np1iu1th', 'kh1eilu0', '4yz8d81m', 'b8nh4y10', 'hwqkl6ya', 'og1nd2bo', 'rhgztthz', 'vkhg20he', 'iggo3ufb', 'd37c9x4c', 'p2iazl7z', 'zx3i8mpo', 'vwh6m7l9', 'cz136auz', 'h1s7b0qx', 'ohx6cnag', '443dz3q3', 'mr5yhwyy', 'qva0jt86', '9qz1oq0k', '15pn9yxe', 'm2txctft', 'cl5cyuj8', 'f7s41g4z', '2d6o7cvz', 'j16tz1ib', 'pyoq30k5', 'nw1qt3xl', '46qcxs4j', 'hjg3kzas', '3g83bfuw', 'c2bwky6e', 'xul5ydao', 'mqd4ioj4', 'sgj806vv', 'czex5g86', 'x8t8a2r4', '3jhh2oo1', 'w5lavhgr', 'ipwuoo7u', 'cnj1og8s', '0r8gvx86', '2hqo4v2x', 'x3bbhi3d', 's2fkyvdh', 'q7izbxd0', 'ou3wj4rz', 'temnlbv9', 'onmzm888', '009k5htk', 'a7m0d8i9', '96tsggzl', 'dtshy2mq', '75dn7si3', 'ngln7fvq', 'u2hoxip4', '1cxnta8c', '87vuavig', 'hbpk850a', 't0m4mnhm', 'pt4eplfi', 'z9hbu9xi', 'j2v9nlav', 'xaqkd5xd', 'j2kpz2cj', 'td8q2jkx', 'suvujub0', '0bfjyum5', 'v5egtgdo', 'hx6e6gid', '27eyjedf', 'okd2qlek', 'gjkdm90a', 'ylcr2vma', 'o976nab9', 'vlaq3sdq', 'vjs0gvb5', 'a4mtee9e', '6e0u9h68', '90kj6kua', '5iqtj3db', 'i8dcdy8d', 'fd265oh7', '73k0dig5', 'hjrmcw98', 'pp1owsdm', 'spxu93n0', 'fkfvau5f', 'vnp33wdc', 'f04wk9w9', 'hp6lwpn5', 'oxm2ldgi', 'ymcvmgv0', '921kvrrz', 'vk0p2kul', 'yvc4432j', 't7ji0x9n', 'yvab1c2f', 'l6guql8q', 'an105nip', 'nxnzga1f', 'r73jlgtf', 'qu2o2iu1', 'pkbo2lbk', 'kng4z4kf', 'ulcv2x36', 'ad3i9wgj', '51qbdrxj', '5vwu0f0c', 'maf822bd', 'ox1m6skh', 'dd05fjtj', 'wnalsdx4', 'oi2clv0d', 'v8lpw4r7', 'sknqk08l', 'y331nw54', '769q0q77', 'b50gptdz', 'sb6bevb2', 'lc4qvvou', '9gqu9r0x', 'hpnyfdhe', 'hhubtrme', 'j2xn6260', '1vozp2o2', 'd8jzbop2', 'th8p0j3f', 'mcbmokn1', 'ev8zvdar', 'a172od10', 'hjct9n64', 'gtjftp0c', '91c7n9dt', 'm9q4av7g', 'bwhfrk5k', 'rl629sdw', 'a9o2dvke', 'i51gcu7e', 'eyobjalw', 'avlinspk', 'dcphc24m', 'ar27zwmr', 'yjcb1stx', 'e0dq1t1v', '3dxt3qqs', 'if3wav59', 'fgw4wps2', '1zgvzpwv', 'lp3o0ikl', '3lujp0oy', 'g0clai34', 'byxuruk1', 'bj6q86hb', 'zlh9eiup', '4enhsmsh', 'jkzd6ktl', 'v0j5oq74', 'zt91o80q', '20jys91a', 'ue591g98', '2qqaisix', 'my5cwjkg', 'fhcdt85e', 'ubw2yl06', '3b3dyysv', 'qu4tn1by', '5xn5yiaz', 'ojh3vgrb', 'ygnjuxwt', 'p45x2kdl', '53y370km', 'o58kbiqv', 'oqg790gj', 'jqmz0mkz', 'rwz1s8tn', 'q66dyx98', 'qwgde9uu', 'h10ye46d', 'peql7vr5', 'rzitixa3', '6akyxdil', '6zyj0ixp', 'eioy4rhn', 'zkfgj825', 'wjvcqxu0', 'aehehpha', 'ur1vaain', 'gutwumol', 'qvqkw7iu', 'yq4kfmia', '8mt27m8e', 'r0509r3x', 'vxbeazjk', 'g4zti8an', 'xl8i5t3l', 'y13ca3ne', 'i8drk84d', 'ajnyt1km', 'mdqvm0qr', 'ip2b4157', 'ur6stkx0', 'vm86d4mj', '5q40uttg', 'zy4f5xdu', 'nreqluol', '3868zydj', 'wusznlzq', 'pmog8726', 'dfl2nr5c', 'r6n7i2gf', 'j00x29vf', '4svaq0at', 'wvvu1uyd', 'i4t84krs', '84oukq5g', 'ciihfpjd', 'frllb1ap', 'vbwgeqkc', 'xhez8byy', '81rr4bbu', 'ibop0bf7', 'pvwrzwzp', 'mgn5x4f5', 'xt71w4nr', '5emihtwr', 'hap1xat8', 'v9ir8mbd', 'usyfeosd', '9fnm8idc', '9leljj8j', 'kmyoavzi', 'nyocnvvj', 'wkaioy19', 'zqqlujwz', 'of372xvu', 'ig728p2a', 'm9d8ngo5', 'ac1vobj1', 'm6bwfubs', 'qtr90z7x', 'qpu7ug5k', '5uwzo304', '1ksz6nmx', '42sa5nrb', 'qoynj85f', '2mceonc1', '3k3qc7i1', 'qvyoz2u6', 'oymg1bba', '9qw7morm', 'i43xer43', '433zgs66', '8ubmb52o', 'vi88bpl1', 'k6z4yn6m', 'txdes9lc', 'gwtmtz4m', 'cnnytp2v', 'lw5u0lwi', 'ozh11en4', 'fcxsv6zu', '7ozgop4z', 'cr0zskcw', 'dcqcp0s2', 'hyx4pn81', '22ec0rdl', 'v6zzrj3n', '9pav7nnw', '8mpbnr0r', 'spi2v0gq', 'e9s4d9u8', 'tvbnv5gz', 'h6g8erxf', '14gq60b5', 'ml3p6i9q', '6gphg65k', 'eenf6ril', 'w7q5o8wc', 'f8x8giqd', 'xk2at7yk', '3utjd3zj', 'ye0fvrur', 'rwcaex4w', 'p3r07i7f', 'fe4ubj1n', 'nt2940ni', 'weiqdxi7', 'qs0y7yc4', 'd7n5wpha', 'ykw4n7p7', 'xiajv4gj', 'tip28ifz', 'hf0iv1ag', 'km5qyl9f', '26e1p4rd', 'dq3p7lky', 'j979o3zc', '2xinqa6f', '8b70rn1i', 'atimt1ko', 'kemp42fw', 'esogd26s', '3nrc8djc', 'oxi658l9', '97c06j9g', '10vx01qv', 'cspub1sp', 'uy8tm5j5', 'm8b574h9', 'fwifxuse', 'okb61ueg', 'v6oq8xas', 'uo5b7etg', 'p18pfwcz', 'nmdbqukk', '0bn0d8vm', 'd08miv07', '68bcq2wr', 'payjduek', 'g6n0xt63', '3aldwdkl', 'll42sgqc', 'o34afjmq', 'tvwlvoz6', '41w0j7a0', 'rhtlyz6t', 'jk2e9wy0', 'sz9qt9vi', 'ya8yit1w', '1tttgnfv', '8fz5dvax', 'jbt5f5yp', 'sihtclpn', '90w7o4de', '3ttrptzo', '9vcm6rj8', '3epj70vy', 'gaepyggp', 'pzivb0ro', 'mtu2thj8', 'cioz31io', '1ea1miqd', '4in24g2t', 'vn405khn', 'u388t8kf', '422gft08', 'rwxspzfs', '8yiyj28w', 'h5ofzjqs', 'cg7wqc0q', 'rc9pejmo', 'itab7zdt', 'j167ywuq', 'rgin7209', 'f7vl8sgx', 'h5l7ahg9', '4ldapxe8', '5d6cddud', '28xl8bbe', 't3vea9uv', 'iexq4l7z', '3lbrfsrh', 'dz19s1jj', '3gqqsb3x', 'ac4fouk0', '6hjxmecx', 'zymjhm6c', '56zica13', 'zu9n95hc', 'nvk4g76o', 'rlvtcyms', 'sweua9ph', 'q6kw8h8i', 'nkyipfu4', 'v9qkzkzl', 'd07repwj', '8bvjpwvv', 'l1qm05pu', 'otpjh7s2', '97xah833', 'x9bxnrtn', 'idttsu4v', '72mjpcp9', '41h0768n', '446b5k1w', '4ynyeh73', '5y8gqqm4', 'n09zq6an', 'b9jg14vs', 'g5d8v9lj', 'p7f1yyf9', '4y579qzy', 'y3pydjg6', 'hvo5j6a4', '0n37jgjv', '32xru8ul', 'xne08vbd', 'ghl62oio', 'c90m66ra', 'ctis78ns', 'dyq9busp', 'v10pg7oe', '7ttb8wkd', 'garchpin', '34q17znp', 'w6cx035t', 'g33uwwtq', '27f9241x', '8hau76g7', 'ac84jwxm', '2zds64bn', '7osrj7db', 'qqkd266u', 'zmi3z6xu', 'd5t7wg4y', '6yr14yk3', 'vp4qnldi', 'u47qtzo4', '8dl4d2se', 'rc8o8fgj', '2mnx7itu', 'kdou7za8', '5rh7vl3c', 'fqj6xff6', 'fgzrky84', '0w1fj0wd', 'pnl9th2c', 'ms5rz8ho', 'o88tv2j3', 'dq226o63', 'u2c6rh36', 'd4434qva', 'xofwk88a', 'rnk9srk6', '4bmb5lfa', 'h2eepvvg', 'k09vcq96', 'ryvsgj8x', 'bwa4wz94', 'pvfsn8hl', '32ykxt0o', 'vverad5j', 'gyrnnrox', '4d4l6mzl', 'i3otm50p', 'p80lfvlv', '99r8knkt', 't8jxqbso', 'vx4eq7qs', 'ftvnb09n', '5csizydl', 'm89u87tj', '2z8ujy53', '87fccpf6', 'cad6pb3n', 'qd6k8u5a', 'q99o5vnk', 'g3a9onb5', '8ut3kio6', '4juvlqpk', '7n7sv82l', 'bxrc8nbc', 'j897sql0', 'icbx5sii', '7mzqjnu2', 'auuyjct6', 'sd3lqg4h', 'srxo19hg', 'cze6ea9p', 'usq41tnk', 'lpnt3ug1', 's5ymgxip', '2ny3hndu', '9j1da5u8', 'jgssstn3', 'bmkssp9g', 'gn1kz1y1', 'oprpgcyr', '0y704vju', '7wxv6e0i', 'vizmbkhh', 'jt0y44d7', 'h7wfxhrq', 'ymsl7snv', 'sf7s2x2d', 't9z2ynt5', '6yf5vnld', 'ms1kgdi8', 'jjzz9xwy', 'gw0tgru5', 'xtx1g5n0', '58s5rfub', '4cjlolp6', 'l46stjow', '2a77stf6', 'ftleu2u1', '7z3wgtpo', 'xaf8ydf2', 'lxcbqvx6', 'w7o52qk3', '7ue0ew91', 'czv0dx5a', '4yymaw15', 'youuvyxe', '6ramef9m', '54blikii', '51n8zh5s', '178i7hzd', '0t4p9d3g', '21cw4w46', 'jyqg1lu3', 'e5kfhbif', 'xmp2zehi', 'qz6q1rfl', 'xaowbavb', 'uc2960sw', 'uu8nb7cg', '2suautrj', 'ch23caq0', '7lcqf9up', '8hnqebb5', 'pufcn56j', '9ua3b8d4', 'xmtgooly', 'kywpqq2c', 'x03wjbtx', '1tn3a4qs', 'o3zopzkq', 'i7icvsg1', '0gwjbt3l', '5r9p44uh', 'kgfggt53', 'jb5cn3qp', 'p9opvoro', 'gjjn8y08', 'pdr5oda9', 'vkxrbav7', '64q9e4ek', '1gf8n3lf', '52xia4kv', 'qbldmef1', '4brpb22r', '120ikm7m', '44uzwwpa', 'w2fjy5od', '2ygv295s', 'r4fqdfyw', 'pvugij9l', 'nrxg9ndo', 'rmpoxuua', '5yxr1j6b', 'a2wmk9an', 'b1iyr42n', '8r59z4u4', 'oa3hg76b', 'upbb9to1', 'kk4yul7h', 'dgk8vpl5', 'n2tl2d1n', '17kf1q3y', 'exchawh0', 'nm0xqwh2', 'h3z0cdkh', 'sxux2ujo', 'ujf6v8x0', 'yldxajbx', 'hx7657i4', 'n9g1p1ux', 'a71mddjt', '7l8lqi1e', '6bh08901', '65q3jd2g', 'dlhcih2w', 'q512vvj8', 'aoa92919', 'hcensosk', 'znfrnosz', '2tz3wyb3', 'clyhurjl', 'xgsn30en', '6tuph2gm', '40wlviqk', 'p3sq99yg', '752xtrcd', '1i8u0ppr', 'i00lozh0', 'ykz8kdrn', 'asy5wanw', '99lpdu81', 'wd2zkgbt', '63wliagj', 'da71febn', 'h2mrhnpw', '3nb0ol6s', 'ok9ziazl', 'odm1asa9', 'mm9fhpuo', 'ig9t3brm', 'maiifhwy', 'tj8bo1ef', '7m5i1l16', '8cplu9mi', 'n7wj4qa4', 'nq6qs5ba', 'annw96an', 'qsoxdtfh', 'kdn8xu7d', '3ds6k08v', '7j2x9ktj', '7c1cmazh', 'p1idx9lz', '3rnr8lpi', 'noml0axi', 'dap0zo2m', 'qpfxu44k', 'lwyjgq5v', '3ezbzl6k', '2rd8cqhg', 'nl9mmqau', 'ldzgjjv7', '0ks9uw31', 'xvvlgwk6', 'j4b8cmc5', '94h6rcih', 'z994ks17', 'diob7j9h', 'k40ef17d', '4v3mebyw', '5s53101f', '3iba6860', 'uurjhoxa', 'ruuz0ahn', '3t52xxhs', '62jstj7p', 'c5aobw75', 'akdpk9bc', 'zmmef8gj', 'jm60nxc2', 'aou2mfe6', '4kjaequt', 'p9pqxh0o', 'xlbnmqr6', 'sx1cewy1', 'ddojms4y', 'utt2quge', '1upboqcr', 'uz4nh8xd', '9sc000kv', 'b2hl53hf', 'xdy5jpaz', 'bu2vs9s0', 'js9vvsud', 'ar828fxo', 'n6gour6r', 'zum861la', '1y7irxr8', '4kvjmhav', 'sny9w9y8', 'wsjbz2be', 'v2uf0apv', 'ncu0glzu', 'usvreyw4', 'cd5lgohu', 'eb82rq5k', 'kcjpqtdo', 'k2ake1rg', '4k1cl05u', '3sah78lc', 'jrdf781h', '3m84bcws', '0xqfej85', 'bnhkw9ra', 'zy7fgjmy', '471wplvu', '4410o8eq', 'dd2sy4zg', 'fnply3ne', 'vqeagrsf', '44wxplmy', 'nyvmtthx', 'sdzk4k9v', 'vhtux6um', '3raox8db', 'gws16hfr', 'g8pedw5j', 'x7mam4y3', 'dppn4dxj', '2piml3jl', 'bd31vmsn', '59uidbl7', '1wtwj43j', 'kuamhfng', 'xog2y6wk', 'i10q7muk', '361jmzhd', 'ep23hi49', '298foqr6', '2j8n9rdh', 'nada82w4', 'h9ojgy5q', '8e8q69jf', 'ztvvh6wx', 'm31mnzps', 'ahb2q953', 'xfmk30u8', 'xus3wo5m', 'letfyl4i', 'eo2w6cpm', 'nkrp6ghr', 'ygi0t786', 'pnc63euh', 'ptbimeap', 'tyqjzgk2', 'k98jgyfl', '19kof813', '7h2ufqg7', 'dfdo7uuo', '3avh7rf1', 'x5iue8us', 'o3413hyr', 'fxvw603y', 'f49uknpm', 'm2fwc619', '34r5ojpw', 'dr0tbvgd', 'sepdptf1', 'urxr3xgp', 'jwanhgi4', 'ze6soygu', 'w6zx7b0u', 'xerdn67b', 't6kiolap', 'rc6mw4il', 'krpfv6am', '1m5h3krw', 'qylm0koe', 'mm3ydvsb', 'fqc01jvx', '366cn0c4', 'dkdg3n7j', 'xzmdgj60', 'p925fokp', '0rbs0yuh', 's7iae09l', 'eg9w0ff5', 'rcft93dc', 'tc4vulou', '4b3zc59x', '70eyenq2', '81nnlz6z', 'jgzodnfe', 'xo7nu4zx', 'wvshtdr3', '4r9fwu2l', 't572r1kw', 'oaugkgx4', 'zlc0dv0a', 'cy9md5rf', 'g5n87inp', 'j16kjpbi', '850bvb9t', '1sbw1zhu', 'nxw5k9y1', 'an909brb', 'whmyqlj4', 'ftjcj2fj', 'q1pq4ew8', '460m2wf6', 'ljfagqls', 'dmzhywb9', 'k0dccpul', '3xo5xueb', 'cbmlb0hv', 'am9x7nvm', 'umfzrawo', 's8q56jv9', 'qc5stoap', '7x5xd3py', 'dc82m6ep', '66vnwrvw', 'knspa1vb', 'an16dgpb', '6n0sn5lk', 'rye3ru1g', 'bqzy4ozn', '3ywogqnm', '7ti25k3e', 'r5hy6gow', 'q58ued7v', 's18ltbv5', '7y8ixx74', 'o9qsd633', 'a9r422lc', 'jdvyrd1c', 'xiwsk0rm', 'wc3gas1k', 'ut28u8gu', 'pj5pf0lu', '8cpt5810', '2t09zqqi', 'a2mjkz83', 'ceds3mpd', 'h6v2i0fs', 'tk8t6v4z', 'm32x4498', 'rj61mzqj', '6yvdv1xk', '7n6dnv6l', '7pxcim3b', '0ylgwkjf', 'xh41koro', 'w7r71ngw', 'bq4cy4rn', 'uvor9rts', '04oa5h38', 'pll5d3mq', 'y7s9jorl', 'lz6y05bs', 'wgx0c0f0', 'l93980xm', 'uzd729zl', '2g9ggwog', 'giqtl9cp', 'p6bktqvh', 'j4rzoins', 'q6sboyj1', 'v5wpgg45', '5zaocebz', 'h5oa8krq', 'sd4nwwfx', 't7fvn3fl', 'ntggifbr', '369oy4it', '2zp88mni', 'xnm9r0cm', 'jsuuckg6', 'bt22c3c3', 's3hy3yug', 'g5y3qzuc', 'ga4du4k4', 'llmmlplx', '1qtlymdj', '6hwvcmf0', 'f6mlvl56', 'l918rflt', 'sasspze6', '1k753udw', 'rl5sd2nh', '4k9m9xe4', '9np7k2ku', 'gqp8ndjx', 'jjoik317', 't6xtg8n9', 'gl0wiyt2', 'czn2inyy', 'nz9onnq6', '1sle0d1j', '4mxt8vm2', 'cr3tbw9w', 'xaphn2x9', 'ft6ud6za', 'bjv84op2', 'fz7a8c3l', 'vxdpx9h8', 'zn5zu9hb', '9m6pyebw', 'kswby0it', '7l47tttd', 'scvo7lay', '1a4xj295', 'whtwovug', 'mcyvnxji', 'zmxwjfaa', 'lflt64qj', 'vzel6r43', 'xzusv5zj', '8w3rjteg', '0002xs6a', 'm7ulxcck', '7jveg7d8', 'i4j8h8ce', 'ekpywzej', 'ihwnvhx9', '185rrhlu', 'gcxyf7bd', 'kcs4pav3', 'b7j6bwv7', 'lysfius5', 'u4p5o0qx', 'ebel6oxd', 'zf98jzdn', 't1lrmahg', 'z3obqltq', '6iwpy3ck', 'r6tgizu7', 'vxkhn4u9', '7euhtr36', 'yvsyo4l9', 'sqjngqk5', 'epxbv1hy', 'm0bmcwno', 'piffgmky', '20ht45u5', 'ojfp7yey', 'qoh1kp5g', 'sltazgbr', '0ydxbgj5', '4xzjf16q', 'mfglfgdw', 'ar4lki90', 'iz1vat9p', '43id00fg', 'acz74ufk', 'hfcdq33h', 'yyb0ra1p', '5eunbdoq', 'modtthxx', 'xx2d1s41', 'j4w4uy1k', '2cdqdmsj', 'sox12zfo', 'prmo5z2t', 'aquvpc32', 'zx2ruyjt', 'wd512k9l', 'y5vws157', 'bqy6airw', 'aclmmnhd', 'qnac92vu', 'vznce8nz', 'm0snyip3', 'ccdzqs0c', 'pstz2xme', '6ql9ckqi', '7kirivh0', '6at89rig', 'xp20uyin', 'e01mkk64', 'nkv4sutn', 'hmr2tpp2', 'vo6nvv71', 'k4ytx6fy', 'graj71lm', 'j5hlhsq2', '7oazhp4d', 'wgpu4dyw', 'vru71761', 's5q6pxhh', 'f3n73d0j', 'rei1e3jr', '4ov7r8p5', '211al7ta', 'mj86h70i', '6hep2lin', 'p9c7zj3p', 'e3ulhon7', 'jrnzqcs5', 'vwguwrf7', 'vigv1snj', 'hpzykj2r', 'sb2srjjl', '2j3g0lez', 'ol9ilsxv', 't2lbqivx', '9x53lq66', 'zs2uaqn0', 'sq38stot', 'j84wwr8o', '2ma5zr4j', 'qzfxgtp3', '4lx3lif4', '0q0sxm50', 'iynafvkd', 'mal0112q', 'pd8m515q', 'od3jymza', 'kyxvu3re', 'v33kum2m', 'd5m2yppx', '93skfa7h', '7nzqxca7', '0n7i771h', 'ncbbno22', 't5z8w7wy', 'rds5by7w', '80vpn72e', 'z7i9bny5', 'rzvwqlal', 'h0ma163u', '5czywnbr', 'kcp6v61c', 'k6l62cpl', 'tg7bi1wl', 'avpx3evd', 't4fvib0h', 'xwl3u09h', 'v71436np', 'rdhehxqk', 'oi17m6gu', '801akw6q', 'pxxxbfm1', '0dyf8bx9', 'ce8ng2et', 'ks9ilcua', 'psg0j39k', 'i7721nvp', 'zi109bg1', 'd36evkis', 'eoovm5gh', 'gvo7pmsw', 'oi7fm40x', 'te02u0d9', 'cjxuvyh9', 'ny590seq', 'yiy567sk', 'yecaao6b', 'w7jbom4z', '5is3qkay', 'rfyo308q', 'u3ylpzxe', 'ad9ceufy', 'b0mru37l', '6kcz8rp4', 'rtjnbn4j', 'zr4xw3ph', 'gafrbkbt', 'n9qdnj00', '9puxj1h9', 'alnq9gmg', '6foz003n', 'kdfm10gf', 'uvk84hm1', 'qgszh2dr', 'nxdnp1ra', '3uz0v29k', 'se0sq3c3', 'z403zm05', '967mrrhi', 'un0gxtph', 'dbksjlk1', 'y3cp5wza', '649l3orp', 'u4y5bydx', 'pa959xmu', 'c4v13t13', '4k4d2zl6', '1wnvgl4j', 'mabdc6zk', 'dr92aqnz', '174q2pjw', 'gbcmrkfr', 'd9mbnz24', 'wdx9636u', 'iv96kndb', '0qxzpvm2', 'xdo0cyue', 'xtkulyac', '5gb71h8g', '1rfmdgxu', 'x35jq9qs', 'yuz510tj', 'qb88t369', 'vc79j8vp', 'xlrq8grw', 'butuv3n1', 'ubn7vren', 'fh6a6i1o', 'lhj7cd8t', '0xky3of4', 'mexfkial', 'yts4xppb', 'd1uzjd85', 'big0pxyf', 'g2x2rj43', '0rllmbgi', 'vk4ag44m', 'tmeg7vou', 'gdjmhjkb', '0fegsm1v', '1t6w08yt', 'zf1w8ksm', 'hl0td1wd', 'nf5w5qi0', 'h4f1qyvg', 'zrnczve3', 'vxwhlq9s', 'ehw61gu6', 'ilmmvjxy', '3ozef29x', 'oi2897w0', 'lqjtywki', 'sridr9wx', 'q0f96ojo', 'ehp0q0ry', 'fntdfq99', 'pxvlwe5v', '2y5nsclo', 'zl7repqz', 'wov2ewtq', 'cyxuz9qq', 't0lgfbt7', '0yabira3', 'cz1gx9oj', '09rg4mx9', 'm47miang', 'wxf928q9', 'zl299g3l', '1x58h3os', '6pgzlr8k', 'p8b3ini4', 'pckii4vh', '2e5ppqub', 'phv2sdta', '8lnmua57', 's5w293cn', '53hxvloy', '8rhvni4t', 'wfljfiut', 'nu77ay9e', 'wy5mzr00', '1y4h92b7', 'egccic93', 'y4yw33a4', 'mo7uilzv', '9xelf8oh', 'jq7h7ylq', 'nzxjjh9f', '7qdlkcro', 'fmckn6xl', 'ugirvxk6', 'yhaawaxm', 'pacy48qx', '6thu0dlp', 'xa6ch9ct', 'ks8iubhe', 'bq1g5g6p', 'v8xnj1je', 'l2t1xa3i', 'divgkg2s', 'b820t16v', 'b57dcpgs', 'p2i1s65b', 'hort1ssa', '0mobdg2p', 'gtkx1r4a', 'awzdm1yy', 'h5rtehxb', 'lc2uw5un', 'qzc72dcy', 'tcmn1y2s', '4doy4jv5', 'nuep8nim', '2q9el4cv', '2oo1jj3j', 'l5ihrxsd', 'u9zbopdh', 'teq5txm9', '7xd8palb', '2vehzf5t', 'rd61t7ln', '3wpbsy2h', 'b4ssn65h', 'm9d78gjv', 'kocj6nqc', 'budyo6no', 'ncrwpte9', 'q04eim1f', 'b29vejwy', '5uov1suw', '4u1g2bgf', 'db4gm90i', 'nxonnj2n', 'fe1oosoa', 'xorhlasc', 'm4jo053m', 'vp22xd8p', 'oygen8m6', 'pr8jcf7i', 'r8vsizc7', '1sphepoz', 'f5yezdeq', 'bzi38q5k', 'a6jm66bq', 'hz9bs7qf', 'p22goxm3', 'y7gj12bu', 'mgbin5k4', 'd0p2nqb9', 'y74liuon', '3q2jju2u', 'e6i3eou9', 't49q48y9', 'mdmqcvww', 'h77uyv6l', 'rzn4yur4', 'r1fxadfz', 'bx9xm6a0', 'q01953r0', 'pyrittj8', 'fwid0tyd', 'eqfj9iyr', '9qsqj91d', 'qx1dqd3n', '2y1oh9sk', 'wei11iyv', 'nvussleq', 'qp68b2cy', '4wx5y4tx', '78o1j2t3', '8u3hmkgl', '5944n58e', 'p3mb7r0v', 'sokho462', 'jt2axpji', 'sfnqz1yi', 'uil0u41k', 'g0rx5wgf', 'rzazvmj8', '9yedp5di', 'eoxzzhdd', 'lkfqv25y', '4hlvrfeh', 'ozatry2n', '0insc6ua', 'e7czpd1y', 's2ksgg67', '365z3qxz', 'kmmfnz8v', 'rn4gm3sc', 'wq5x6mrf', 'euw8dv8f', '7k05aoya', 'f24o9jx4', 'v5a2jeg0', 'jgt3vap2', 'qdbo3u55', '3qucotv5', '0u0znorq', '5yfffep4', 'wz0kzv3n', '5yiv6p2u', 'iiokdpjs', 'vfqoryy0', 'xi1mln06', '0zz7on3h', '3x6nwd91', '3kn5g766', 'ha897bkh', 'e795m6tk', '3eph4c57', 'zpcjfj8c', '6vmoupen', 'm4liogod', 'hzfawetp', '0kkiw77i', 'pot4hbep', 'jol1lmh6', 'jr8l6rlo', 'yww098fu', '8s4bsnlw', 'm0x5sbo3', 'wh8yclcm', '88hibsk5', 'ek6a82km', 'r60j1gm8', 'nmmimc71', 'jegzg0tv', 'f8exyewx', 'jxkq15ab', 'ok11dla3', 'vbnnm9qg', 'j7vlnlr6', 'ik58kfpm', 'brpngt14', 'osz2vew3', 'w8m69hlh', 'yvxay6dn', 'okmn7rzq', 'kdf654z4', 'gmxei6n6', 'mhxrrnyh', 'hgoumvkz', 'r1f6d1za']

In [5]:
MACRO_TOPICS = ['Coronavirus', 'Public health and epidemics', 'Molecular biology', 'Influenza', 'Immunology',
               'Rotavirus', 'Antivirals', 'Clinical trials', 'Testing and diagnosing']

In [6]:
RESEARCH_ASPECTS = ["Background", "Purpose", "Method", "Finding_Contribution", "Other"]

In [7]:
av = ["Background", "Purpose", "Method", "Finding_Contribution"]

In [27]:
num_covid_docs = 3000
num_all_docs = 3000

covid_uids = []
all_docs_uids = []


with open('metadata.csv') as cord19:
    reader = csv.DictReader(cord19)
    questions = list(reader)
    
    covid_docs_count = 0
    all_docs_count = 0
    
    while(covid_docs_count < num_covid_docs):
        random_doc = random.choice(questions)
        is_covid = bool(re.search('COVID-19', random_doc['abstract'], re.IGNORECASE) or re.search('COVID', random_doc['abstract'], re.IGNORECASE))
        while(not is_covid):
            random_doc = random.choice(questions)
            is_covid = bool(re.search('COVID-19', random_doc['abstract'], re.IGNORECASE) or re.search('COVID', random_doc['abstract'], re.IGNORECASE))

        covid_docs_count = covid_docs_count + 1
        if(random_doc['cord_uid'] not in covid_uids):
            covid_uids.append(random_doc['cord_uid'])
            
    while(all_docs_count < num_all_docs):
        random_doc = random.choice(questions)
        is_covid = bool(re.search('COVID-19', random_doc['abstract'], re.IGNORECASE) or re.search('COVID', random_doc['abstract'], re.IGNORECASE))
        while(is_covid):
            random_doc = random.choice(questions)
            is_covid = bool(re.search('COVID-19', random_doc['abstract'], re.IGNORECASE) or re.search('COVID', random_doc['abstract'], re.IGNORECASE))

        all_docs_count = all_docs_count + 1
        if(random_doc['cord_uid'] not in all_docs_uids):
            all_docs_uids.append(random_doc['cord_uid'])
    
print(list(numpy.concatenate((covid_uids, all_docs_uids))))
#print(covid_uids)
#print(all_docs_uids)

['ca5twlo7', '9b5cuyae', 'kabpuznc', 'un6u1suu', 'pgyuk9z0', '8cq8p3iy', 'chmnogt6', 'qxso58u3', '1o7vxcvv', 'g3m7ksrs', 'w4i0q6rm', '3dk9wnw2', 'd7jmakcj', 'fo3wl3jy', '78rr4pz5', '04pp0o74', '31wrnk3m', 'a7zith7z', 'i9ret7bw', '2sz7j89r', 'po5o6996', '1m3bqw9s', 'ml673d5s', '8qcy0nl1', 'ks8brz5c', '7hjmwnhr', 'izyjo3r4', '2af5c1da', 'vkoawl96', 'uy97jjyi', 'qw05apnf', 'fb4hm4b0', 'aurhahaq', 'a5deq2a7', 'htbgb8r3', '4ifwiisy', '4g6wq80m', 'fu73pzgg', 's28lfwtj', '99jlejuv', 'uv4fvlos', 'pjr185mj', '20muep3u', '8xg6nn5x', 'kc546u59', 'qdeaw0uz', 'bnjupaik', 'n4dgqo73', '9vtauu52', 'sijh2ouj', 'aklipjm4', '13ym1n0k', 'r64d3fv8', 'ekc2zs4i', '4qmajxtb', '43jmvu7o', 'xo1f35jg', 'qfdp392f', '4ipuy7su', 'wrh27ban', '5icvvvp1', 'mm2iwtf6', '3yxrqa22', 'mq5xq46v', 'a8gfr4fa', '67h9z5uu', 'vlx22si0', 'bs3bwzah', 'ei1nkcmf', 'e5u1ymz9', 'edelrr0u', 'xj9md751', '55yf5o1z', 'gy9roon3', 'fzhj0ec4', 'zpmlb143', 'el283fej', 'cjb0daps', 'nf20zqcj', '4wk8gsli', 'btl03892', '6u1zo0f9', 'oihyz5v7', '0b

## JSON builder and topic builder

In [8]:
class Builder:
    def __init__(self, metadata, progress_bar):
        self.metadata = metadata
        self.progress_bar = progress_bar
        self.entries = {}
        
        self.nlp_sentences = spacy.load('en_ner_bionlp13cg_md')
        #self.linker = EntityLinker(resolve_abbreviations=True, name="mesh")
        #self.nlp_sentences.add_pipe(self.linker)
        
        self.zsl = ktrain.text.ZeroShotClassifier()
        #self.linker = EntityLinker(resolve_abbreviations=True, name="umls")
        
        self.rac = ResearchAspectClassifier(RESEARCH_ASPECTS)
        progress_bar.value += 25
        
    def return_dict_uids(self, uids=None):
        reader = csv.DictReader(self.metadata)
        papers = {}

        for row in reader:
            if(uids is not None):
                if(row['cord_uid'] in uids):
                    papers[row['cord_uid']] = {'title': row['title'], 'abstract':row['abstract'], 'publish_time':row['publish_time']}
            else:
                papers[row['cord_uid']] = {'title': row['title'], 'abstract':row['abstract'], 'publish_time':row['publish_time']}
        
        return papers
        
        
    def return_docs_dict(self, papers):
         
        progress_paper = 75.0/float(len(papers))
                                
        for i, (uid, paper) in enumerate(papers.items()):  
            try:
                abstract_nlp = self.nlp_sentences(paper['abstract'])
                sentences = [sent.text.strip() for sent in abstract_nlp.sents]

                classified_abstract = self.rac.classify_abstract(sentences) 
                    
                is_covid = bool(re.search('COVID-19', paper['abstract'], re.IGNORECASE) or re.search('COVID', paper['abstract'], re.IGNORECASE))
                ents = [ent for ent in abstract_nlp.ents]
                umls_ents = []
                
                for ent in ents:
                    umls_ents.append({'id': 'A', 'name': ent.text,'label': ent.label_})
                #    try:
                #        already_added = False
                #        for umls_ent in umls_ents:
                #            if(umls_ent['id'] == self.linker.umls.cui_to_entity[ent._.umls_ents[0][0]].concept_id):
                #                already_added = True
                #                break
                #                
                #        if(not already_added):
                #            umls_ents.append({'id': self.linker.umls.cui_to_entity[ent._.umls_ents[0][0]].concept_id, 'name': self.linker.umls.cui_to_entity[ent._.umls_ents[0][0]].canonical_name,'label': ent.label_})
                #    except Exception as e:
                #        pass
                
                
                #if(len(sentences) > 0):
                #    macro_topic = self.zsl.predict(sentences, topic_strings=MACRO_TOPICS, include_labels=True)[0][0] 
                #else:
                #    macro_topic = ""

                self.entries[uid] = {'title': paper['title'],'abstract': classified_abstract, 'is_covid': is_covid, 'umls_ents': umls_ents, 'publish_time': paper['publish_time']}
            except Exception as e:
                pass
            
            progress_bar.value += progress_paper

        print('== Done Building Docs Dictionary ==')
        return self.entries
    
    def return_topic_models(self, docs_dict, aspects_to_train):
        topic_models = {'covid_papers':{}, 'all_papers':{}}
        texts = {'covid_papers':{}, 'all_papers':{}}
        uids = {'covid_papers':[], 'all_papers':[]}
        build_docs = {'covid_papers':{}, 'all_papers':{}}
        
        for aspect in aspects_to_train:
            texts['covid_papers'][aspect] = []
            texts['all_papers'][aspect] = []
            
            build_docs['covid_papers'][aspect] = []
            build_docs['all_papers'][aspect] = []
            
        
        for i, (uid, doc) in enumerate(docs_dict.items()):  
            if(doc['is_covid']):
                uids['covid_papers'].append(uid)
            uids['all_papers'].append(uid)
            
            for aspect in aspects_to_train:
                if(aspect == 'Whole'):
                    whole_text_dict = {}
                    whole_text = ''
                    for aspect in RESEARCH_ASPECTS:
                        for sentence in doc['abstract'][aspect]:
                            whole_text_dict[sentence[0]] = sentence[1]

                    for key in sorted(whole_text_dict):
                        whole_text+= whole_text_dict[key]
                        
                    if(len(whole_text) < 10):
                        whole_text = doc['title']

                    if(doc['is_covid']):
                        texts['covid_papers']['Whole'].append(whole_text)

                    texts['all_papers']['Whole'].append(whole_text)
                    
                else:
                    #if(len(doc['abstract']))
                    
                    joined_text = ' '.join([sentence[1] for sentence in doc['abstract'][aspect]])
                    
                    if(doc['is_covid']):
                        texts['covid_papers'][aspect].append(joined_text)

                    texts['all_papers'][aspect].append(joined_text)
                
        for i, (classification, aspects) in enumerate(texts.items()):  
            for z, (aspect, docs) in enumerate(aspects.items()):  
                try:
                    #0.85
                    topic_models[classification][aspect] = ktrain.text.get_topic_model(docs, max_df=0.85, lda_max_iter=5, verbose=False, n_features=30000)
                    build_docs[classification][aspect] = docs
                except Exception as e:
                    print(e)
                                            
        return topic_models, uids, build_docs
                        

In [9]:
def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file))

def create_dir(path):
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)
    

def build_files(metadata, progress_bar, aspects, is_test):
    builder = Builder(metadata, progress_bar)
    
    create_dir('built_files')
    
    if(is_test):
        docs_dict = builder.return_docs_dict(builder.return_dict_uids(test_uids))
    else:
        docs_dict = builder.return_docs_dict(builder.return_dict_uids())
    
    #uncomment
    #docs_dict = {}
    #with zipfile.ZipFile('built_files_cord19ktool.zip') as zf:
    #    with zf.open('built_files/docs_dict.p') as docs:
    #        docs_dict = pickle.load( docs )        
        
    topic_models, uids, build_docs = builder.return_topic_models(docs_dict, aspects)
    
    with open('built_files/docs_dict.p', 'wb') as fp:
        pickle.dump(docs_dict, fp, protocol=pickle.HIGHEST_PROTOCOL)
    
    
    for i, (classification, aspects) in enumerate(topic_models.items()):  
        create_dir('built_files/'+classification)
        
        with open('built_files/'+classification+'/uids.p', 'wb') as fp:
            pickle.dump(uids[classification], fp, protocol=pickle.HIGHEST_PROTOCOL)
        
        for z, (aspect, model) in enumerate(aspects.items()):  
            create_dir('built_files/'+classification+'/'+aspect)
            model.save('built_files/'+classification+'/'+aspect+'/'+aspect)
            
            with open('built_files/'+classification+'/'+aspect+'/docs.p', 'wb') as fp:
                pickle.dump(build_docs[classification][aspect], fp, protocol=pickle.HIGHEST_PROTOCOL)
    
        

        
    zipf = zipfile.ZipFile('built_files_cord19ktool.zip', 'w', zipfile.ZIP_DEFLATED)
    zipdir('built_files', zipf)
    zipf.close()
    

## SETUP JSON FILE FROM METADATA.CSV FILE

In [10]:

RESEARCH_ASPECTS.extend(['Whole'])


In [11]:
def enable_btns_setup_server():
    btn_setup_server.disabled = False
    btn_setup_server_test.disabled = False
    
import time



    
title = widgets.Label(value='Upload the metadata.csv from CODA-19 to build files')

aspects = widgets.SelectMultiple(
    options=RESEARCH_ASPECTS,
    description='Research Aspects to train',
    disabled=False
)

btn_setup_server = widgets.Button(
    description='BUILD FILES',
    disabled=True,
    button_style='', 
    icon='gear' 
)

btn_setup_server_test = widgets.Button(
    description='BUILD FILES FOR TEST PAPERS',
    disabled=True,
    button_style='', 
    icon='gear' 
)

progress_bar = FloatProgress(min=0, max=100) 

def selected_file(fc):
    enable_btns_setup_server()

def pressed_build_files(btn):
    display(progress_bar)
    with open(fc.selected_path+'/'+fc.selected_filename) as cord19:
        build_files(cord19, progress_bar, aspects.value, False)
    
    cord19.close()
    
def pressed_build_files_test(btn):
    display(progress_bar)
    with open(fc.selected_path+'/'+fc.selected_filename) as cord19:
        build_files(cord19, progress_bar, aspects.value, True)
    
    cord19.close()
    
fc = FileChooser('./')
display(fc)
fc.register_callback(selected_file)


btn_setup_server.on_click(pressed_build_files)
btn_setup_server_test.on_click(pressed_build_files_test)

#display(title)
display(aspects)
display(btn_setup_server)
display(btn_setup_server_test)

FileChooser(path='.', filename='', title='HTML(value='', layout=Layout(display='none'))', show_hidden='False',…

SelectMultiple(description='Research Aspects to train', options=('Background', 'Purpose', 'Method', 'Finding_C…

Button(description='BUILD FILES', disabled=True, icon='gear', style=ButtonStyle())

Button(description='BUILD FILES FOR TEST PAPERS', disabled=True, icon='gear', style=ButtonStyle())

FloatProgress(value=0.0)

== Done Loading ResearchAspectClassifier ==
Successfully created the directory built_files 


In [12]:
len(test_uids)

5917